# Data inspection & exploration: massively parallel reporter assay (MPRA) lymphoblastoid cell lines (LCL)
Direct Identification of Hundreds of Expression-Modulating Variants using a Multiplexed Reporter Assay - R. Tehwey 2016

#### Information Paper
Modified MPRA to increase throughput, improve reproducibility and sensitivity
- method:
- to detect differences in regulation caused by single variants within genetic elements.
    - not just to find genetic elements that regulate genes
- to detect subtle differences in how each allele drives expression

~30.000 SNVs within a set of eQTLs of lymphoblastloid cell lines (LCL) <br>
79k oligo library included 39,479 oligo pairs, originating from _29,173_ unique variants

- single-nucleotide and small-insertion/deletion polymorphisms (SNVs)
- SNV: single nucleotide variant

__Supplementary info identifying regulatory oglio's__ <br>
foldchange: Bayesian shrinkage on the log ratios <br>
Wald’s test to estimate significance for
expression differences between conditions and corrected for multiple hypothesis testing by Bonferroni’s method accounting for 39,479
tests <br>
"a corrected p-value of 0.01 or less in either the reference or alternate allele in order to call a sequence as having a
regulatory effect on expression."

MPRA: centering the variant of interest in 150 bp of its genomic sequence

Genome build: hg19 - grch37 <br>
transfected the original 79k MPRA library into two separate lymphoblastoid cell lines (NA12878 and NA19239) 1000 Genomes project 


### Import libraries and functions

In [1]:
import pandas as pd
import numpy as np
import yaml
import re
from collections import Counter
from read_config_file import get_config

In [2]:
from bokeh.models import ColumnDataSource
from bokeh.layouts import grid, gridplot
from bokeh.plotting import figure, show, output_notebook
import hvplot.pandas
output_notebook()

Loading BokehJS ...

In [3]:
pd.set_option('mode.chained_assignment', None)

### Load data

In [4]:
config = get_config()
lcl_mpra = (config['lcl_mpra'])

In [5]:
lcl_analysis = pd.read_csv(lcl_mpra, sep= ';')

In [6]:
lcl_analysis

,ID,SNP,Direction,Haplotype,C.A.ctrl.mean,C.A.exp.mean,C.A.log2FC,C.A.logP,C.A.logPadj,C.B.ctrl.mean,C.B.exp.mean,C.B.log2FC,C.B.logP,C.B.logPadj,LogSkew.12878,LogSkew.19239,LogSkew.Comb,C.Skew.logP,C.Skew.fdr
0,rs11548103_RC,rs11548103,neg,ref,"893,1479127","1403,234147","0,637129492","20,7261588","16,12980366","985,1325708","1413,418102","0,51219935","24,23929835","19,6429432","-0,157648996","-0,070398717","-0,124930141","1,197350042","0,803895042"
1,rs2016366,rs2016366,pos,ref,"316,5963861","258,9020248","-0,281146168","1,799520609",0,"345,5067697","401,9284864","0,195118026","0,437249125",0,"0,344054205","0,696614175","0,476264194",NaN,NaN
2,rs2016366_alt,rs2016366,pos,alt,"653,1486362","605,3570511","-0,10474357","0,364373469",0,"627,6529116","774,785548","0,287498498","1,736553001",0,"0,390497051","0,395150428","0,392242067",NaN,NaN
3,rs11102212_RC,rs11102212,neg,ref,"272,6823928","724,1879892","1,276379866","20,90307853","16,30672339","270,606119","663,3877116","1,183784025","18,27620109","13,67984595","-0,182603278","0,057416555","-0,09259584","0,151356437","0,102546276"
4,rs646867_RC,rs646867,neg,ref,"605,41296","595,8964287","-0,023234358","0,213685108",0,"978,7519082","806,4493114","-0,270001952","2,243712031",0,"-0,287120573","-0,179512629","-0,246767594",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39473,rs9621715_alt,rs9621715,pos,alt,"650,5299307","507,2816854","-0,342573976","2,658697903",0,"563,5230719","419,8238956","-0,40199675","2,811763121",0,"-0,089696774","-0,008966109","-0,059422775",NaN,NaN
39474,rs4275_RC,rs4275,neg,ref,"718,7165454","859,4618007","0,250260624","2,474740501",0,"576,8049317","802,8002303","0,455673958","5,422338904","0,825983761","0,167058994","0,269337235","0,205413334",NaN,NaN
39475,rs131816_RC,rs131816,neg,ref,"754,8418617","634,680055","-0,24581143","1,350290946",0,"954,7364882","1064,020915","0,151280773","1,036516688",0,"0,473004693","0,270571387","0,397092203",NaN,NaN
39476,rs131816_RC_alt,rs131816,neg,alt,"827,7035064","840,290995","0,01819199","0,117288448",0,"753,1914084","756,1706236","0,003935368","0,110077204",0,"-0,011146895","-0,019439501","-0,014256622",NaN,NaN


In [7]:
lcl_analysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39478 entries, 0 to 39477
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             39478 non-null  object
 1   SNP            39478 non-null  object
 2   Direction      39478 non-null  object
 3   Haplotype      39478 non-null  object
 4   C.A.ctrl.mean  39478 non-null  object
 5   C.A.exp.mean   39478 non-null  object
 6   C.A.log2FC     39363 non-null  object
 7   C.A.logP       39363 non-null  object
 8   C.A.logPadj    39363 non-null  object
 9   C.B.ctrl.mean  39478 non-null  object
 10  C.B.exp.mean   39478 non-null  object
 11  C.B.log2FC     39373 non-null  object
 12  C.B.logP       39373 non-null  object
 13  C.B.logPadj    39373 non-null  object
 14  LogSkew.12878  39333 non-null  object
 15  LogSkew.19239  39333 non-null  object
 16  LogSkew.Comb   39333 non-null  object
 17  C.Skew.logP    4335 non-null   object
 18  C.Skew.fdr     4335 non-nu

In [8]:
#tewhey = pd.read_csv('data_papers/tewhey/tewhey_table.csv', sep=';')

In [9]:
#tewhey

In [10]:
lcl_analysis.isna().sum()

ID                   0
SNP                  0
Direction            0
Haplotype            0
C.A.ctrl.mean        0
C.A.exp.mean         0
C.A.log2FC         115
C.A.logP           115
C.A.logPadj        115
C.B.ctrl.mean        0
C.B.exp.mean         0
C.B.log2FC         105
C.B.logP           105
C.B.logPadj        105
LogSkew.12878      145
LogSkew.19239      145
LogSkew.Comb       145
C.Skew.logP      35143
C.Skew.fdr       35143
dtype: int64

In [11]:
lcl_analysis.columns = [i.replace('C.A.', 'ref_').replace('C.B.', 'alt_').replace('.', '_').lower() for i in lcl_analysis.columns]
#lcl_analysis.columns =  [i for i in lcl_analysis.columns]

In [12]:
lcl_analysis.columns

Index(['id', 'snp', 'direction', 'haplotype', 'ref_ctrl_mean', 'ref_exp_mean',
       'ref_log2fc', 'ref_logp', 'ref_logpadj', 'alt_ctrl_mean',
       'alt_exp_mean', 'alt_log2fc', 'alt_logp', 'alt_logpadj',
       'logskew_12878', 'logskew_19239', 'logskew_comb', 'c_skew_logp',
       'c_skew_fdr'],
      dtype='object')

In [13]:
lcl_analysis = lcl_analysis.replace(',', '.', regex=True)

In [14]:
columns_to_float = list(lcl_analysis.columns[4:])
lcl_analysis[columns_to_float] = lcl_analysis[columns_to_float].astype(float)

In [15]:
lcl_analysis.dtypes

id                object
snp               object
direction         object
haplotype         object
ref_ctrl_mean    float64
ref_exp_mean     float64
ref_log2fc       float64
ref_logp         float64
ref_logpadj      float64
alt_ctrl_mean    float64
alt_exp_mean     float64
alt_log2fc       float64
alt_logp         float64
alt_logpadj      float64
logskew_12878    float64
logskew_19239    float64
logskew_comb     float64
c_skew_logp      float64
c_skew_fdr       float64
dtype: object

In [16]:
lcl_analysis['snp'].value_counts()

rs115855724    4
rs118026199    4
rs116983424    4
rs112595714    4
rs118159794    4
              ..
rs114015819    1
rs76308922     1
rs10262443     1
rs11761517     1
rs2076041      1
Name: snp, Length: 29173, dtype: int64

In [17]:
f'Total unique variants {len(lcl_analysis.snp.value_counts())}'

'Total unique variants 29173'

Unique values (variants) found: 29173. This corresponds to the mentioned 29173 unique variants in the paper.

Determine columns of interest:
- exp.mean
- log2fc
- skew.comb ?
- skew.logP ?
- skew.fdr ? <br>

Find out how get the values. Try to reproduce results.


**Calculate beta/ratio**

In [18]:
lcl_analysis

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,alt_exp_mean,alt_log2fc,alt_logp,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr
0,rs11548103_RC,rs11548103,neg,ref,893.147913,1403.234147,0.637129,20.726159,16.129804,985.132571,1413.418102,0.512199,24.239298,19.642943,-0.157649,-0.070399,-0.124930,1.197350,0.803895
1,rs2016366,rs2016366,pos,ref,316.596386,258.902025,-0.281146,1.799521,0.000000,345.506770,401.928486,0.195118,0.437249,0.000000,0.344054,0.696614,0.476264,NaN,NaN
2,rs2016366_alt,rs2016366,pos,alt,653.148636,605.357051,-0.104744,0.364373,0.000000,627.652912,774.785548,0.287498,1.736553,0.000000,0.390497,0.395150,0.392242,NaN,NaN
3,rs11102212_RC,rs11102212,neg,ref,272.682393,724.187989,1.276380,20.903079,16.306723,270.606119,663.387712,1.183784,18.276201,13.679846,-0.182603,0.057417,-0.092596,0.151356,0.102546
4,rs646867_RC,rs646867,neg,ref,605.412960,595.896429,-0.023234,0.213685,0.000000,978.751908,806.449311,-0.270002,2.243712,0.000000,-0.287121,-0.179513,-0.246768,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39473,rs9621715_alt,rs9621715,pos,alt,650.529931,507.281685,-0.342574,2.658698,0.000000,563.523072,419.823896,-0.401997,2.811763,0.000000,-0.089697,-0.008966,-0.059423,NaN,NaN
39474,rs4275_RC,rs4275,neg,ref,718.716545,859.461801,0.250261,2.474741,0.000000,576.804932,802.800230,0.455674,5.422339,0.825984,0.167059,0.269337,0.205413,NaN,NaN
39475,rs131816_RC,rs131816,neg,ref,754.841862,634.680055,-0.245811,1.350291,0.000000,954.736488,1064.020915,0.151281,1.036517,0.000000,0.473005,0.270571,0.397092,NaN,NaN
39476,rs131816_RC_alt,rs131816,neg,alt,827.703506,840.290995,0.018192,0.117288,0.000000,753.191408,756.170624,0.003935,0.110077,0.000000,-0.011147,-0.019440,-0.014257,NaN,NaN


In [19]:
lcl_analysis['beta'] = lcl_analysis['ref_ctrl_mean'] / lcl_analysis['alt_ctrl_mean']

In [20]:
df = lcl_analysis[['id', 'snp', 'ref_ctrl_mean', 'alt_ctrl_mean','c_skew_logp', 'c_skew_fdr', 'beta']]

In [21]:
df.head()

,id,snp,ref_ctrl_mean,alt_ctrl_mean,c_skew_logp,c_skew_fdr,beta
0,rs11548103_RC,rs11548103,893.147913,985.132571,1.197350,0.803895,0.906627
1,rs2016366,rs2016366,316.596386,345.506770,NaN,NaN,0.916325
2,rs2016366_alt,rs2016366,653.148636,627.652912,NaN,NaN,1.040621
3,rs11102212_RC,rs11102212,272.682393,270.606119,0.151356,0.102546,1.007673
4,rs646867_RC,rs646867,605.412960,978.751908,NaN,NaN,0.618556


In [22]:
df.sort_values(by='beta', ascending=False)

,id,snp,ref_ctrl_mean,alt_ctrl_mean,c_skew_logp,c_skew_fdr,beta
21491,chr14:107235827:I_RC,chr14:107235827:I,3.481309,0.0,NaN,NaN,inf
8281,rs2548532,rs2548532,2.744009,0.0,NaN,NaN,inf
28221,rs117824191_RC,rs117824191,4.053868,0.0,NaN,NaN,inf
28220,rs117824191,rs117824191,2.490773,0.0,NaN,NaN,inf
4355,chr3:41802815:I_RC,chr3:41802815:I,0.522196,0.0,NaN,NaN,inf
...,...,...,...,...,...,...,...
38052,chr21:27033091:D_RC,chr21:27033091:D,0.000000,0.0,NaN,NaN,NaN
38757,rs117007368_RC,rs117007368,0.000000,0.0,NaN,NaN,NaN
38758,rs117007368_RC_alt,rs117007368,0.000000,0.0,NaN,NaN,NaN
39362,rs114078634_RC,rs114078634,0.000000,0.0,NaN,NaN,NaN


Check the missing values:

In [23]:
df['beta'].isna().sum()

93

In [24]:
np.isinf(df['beta']).values.sum()

42

In [25]:
(df['beta'] == 0).sum()

59

In [26]:
39478 - 39281
no_expression = df.loc[(df['ref_ctrl_mean'] == 0) | (df['alt_ctrl_mean'] == 0)]
print('values that equals 0:',(no_expression['beta'] == 0).sum())
print(np.isinf(no_expression['beta']).values.sum(), "values are inf")
print('amount of nans:', no_expression['beta'].isna().sum())

values that equals 0: 59
42 values are inf
amount of nans: 93


Selection excluding the rows where there was no expression for either A or B

In [27]:
dfbeta = df.loc[(df['ref_ctrl_mean'] != 0) & (df['alt_ctrl_mean'] != 0)]

In [28]:
dfbeta.sort_values(by='beta', ascending=False)

,id,snp,ref_ctrl_mean,alt_ctrl_mean,c_skew_logp,c_skew_fdr,beta
23853,rs112610838_RC,rs112610838,238.836186,2.500909,NaN,NaN,95.499761
37121,rs61016611_RC,rs61016611,923.347965,11.318021,0.140348,0.096483,81.582104
11506,chr7:141469761:I_RC,chr7:141469761:I,65.588204,0.842093,NaN,NaN,77.887119
247,rs7541996_RC,rs7541996,13.247435,0.182934,NaN,NaN,72.416503
7786,chr5:96250649:D,chr5:96250649:D,10.462704,0.195876,NaN,NaN,53.414845
...,...,...,...,...,...,...,...
5835,chr3:39457335:D_RC,chr3:39457335:D,13.842983,824.600393,NaN,NaN,0.016788
5530,rs13319569_alt,rs13319569,0.548802,32.812149,NaN,NaN,0.016726
2713,rs4844576,rs4844576,5.259162,362.419021,NaN,NaN,0.014511
4158,chr2:61459977:D_RC,chr2:61459977:D,0.159948,16.152393,NaN,NaN,0.009902


In [29]:
dfbeta = dfbeta.dropna().sort_values(by='beta', ascending=False)
dfbeta

,id,snp,ref_ctrl_mean,alt_ctrl_mean,c_skew_logp,c_skew_fdr,beta
37121,rs61016611_RC,rs61016611,923.347965,11.318021,0.140348,0.096483,81.582104
32918,rs75530705_alt,rs75530705,336.399370,9.215217,0.000000,0.000000,36.504769
30885,rs117818149_alt,rs117818149,350.427680,16.213102,0.818604,0.548141,21.613857
30883,rs117818149,rs117818149,337.225327,18.017214,0.258726,0.174526,18.716841
32919,rs75530705_RC_alt,rs75530705,614.133901,37.772722,0.661211,0.442854,16.258661
...,...,...,...,...,...,...,...
14931,chr9:34318193:D,chr9:34318193:D,64.340388,310.815694,0.134496,0.093223,0.207005
25835,rs2136751_RC,rs2136751,1.735254,20.676853,0.000000,0.000000,0.083923
10572,chr6:74225927:D,chr6:74225927:D,102.816359,1247.593039,0.611847,0.408114,0.082412
21909,chr15:75287880:D,chr15:75287880:D,2.088847,32.393052,0.000000,0.000000,0.064484


In [30]:
dfbeta.snp.value_counts()

rs878887       4
rs141228635    4
rs10412963     4
rs116961551    4
rs28781888     4
              ..
rs9649052      1
rs74539570     1
rs6764426      1
rs112745252    1
rs56186137     1
Name: snp, Length: 3595, dtype: int64

__Amount of unique variants left are 3595__

In [31]:
#dropping zero;s in fdr
df1 = dfbeta.dropna().sort_values(by='beta', ascending=False)
df1 = df1.loc[df1['c_skew_fdr'] != 0]
f'{len(df1.snp.value_counts())} unique variants left after dropping values of 0'

'3593 unique variants left after dropping values of 0'

***

#### determine active sequences

__Paper__ <br>
Identified the subset of sequences for which either or both variants altered the expression of the reporter. Of the 29k variants evaluated in the original assay, 12% (__3,432__) had an effect on the reporter for at least one of the two alleles (Table S1) <br>
<br>
"Focusing on those sequences for which at least one allele affected the expression of the reporter, we identified those that showed differential expression between the reference and alternate allele (“allelic skew”). Of the __3,432__ active sequences, __842__ showed allelic skew;"

In [32]:
f'Variants evaluated original assay {len(lcl_analysis.snp.value_counts())}'

'Variants evaluated original assay 29173'

Drop NaNs

In [33]:
#Reverse log P adjusted value to P value per allele
lcl_analysis['ref_pvalue']= lcl_analysis['ref_logpadj'].map(lambda x: 10 ** -x)
lcl_analysis['alt_pvalue']= lcl_analysis['alt_logpadj'].map(lambda x: 10 ** -x)


In [34]:
# Drop all NaNs
df_tewhey = lcl_analysis.dropna()

In [35]:
f'unique variants left after dropping all NaNs in dataset {len(lcl_analysis.dropna().snp.value_counts())}'

'unique variants left after dropping all NaNs in dataset 3595'

In [36]:
#Check for missing values
df_tewhey.isnull().sum()

id               0
snp              0
direction        0
haplotype        0
ref_ctrl_mean    0
ref_exp_mean     0
ref_log2fc       0
ref_logp         0
ref_logpadj      0
alt_ctrl_mean    0
alt_exp_mean     0
alt_log2fc       0
alt_logp         0
alt_logpadj      0
logskew_12878    0
logskew_19239    0
logskew_comb     0
c_skew_logp      0
c_skew_fdr       0
beta             0
ref_pvalue       0
alt_pvalue       0
dtype: int64

In [37]:
df_tewhey

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,alt_logp,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue
0,rs11548103_RC,rs11548103,neg,ref,893.147913,1403.234147,0.637129,20.726159,16.129804,985.132571,...,24.239298,19.642943,-0.157649,-0.070399,-0.124930,1.197350,0.803895,0.906627,7.416455e-17,2.275395e-20
3,rs11102212_RC,rs11102212,neg,ref,272.682393,724.187989,1.276380,20.903079,16.306723,270.606119,...,18.276201,13.679846,-0.182603,0.057417,-0.092596,0.151356,0.102546,1.007673,4.934880e-17,2.090037e-14
6,rs112338151,rs112338151,pos,ref,311.022339,938.600426,1.454065,35.570382,30.974027,441.094084,...,46.863102,42.266747,-0.149852,-0.070538,-0.120109,1.108686,0.743469,0.705116,1.061629e-31,5.410694e-43
14,rs10910099_RC,rs10910099,neg,ref,1565.369298,2410.813917,0.609195,18.726474,14.130119,1597.275307,...,4.277202,0.000000,-0.319983,-0.442277,-0.365843,2.609976,1.744575,0.980025,7.411069e-15,1.000000e+00
17,rs61731104,rs61731104,pos,ref,787.310108,1106.543203,0.472118,7.158719,2.562364,851.116339,...,2.071566,0.000000,-0.426533,-0.181428,-0.334618,1.448329,0.973007,0.925032,2.739278e-03,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39435,rs6002380_RC,rs6002380,neg,ref,1021.939757,1393.401924,0.436551,7.957672,3.361317,1410.403292,...,3.737057,0.000000,-0.072219,-0.194436,-0.118050,0.225938,0.152430,0.724573,4.351945e-04,1.000000e+00
39454,rs12165508_RC,rs12165508,neg,ref,905.677775,1186.698469,0.370023,10.844473,6.248118,602.818906,...,5.622711,1.026356,-0.004909,0.002897,-0.001982,0.082316,0.061067,1.502404,5.647837e-07,9.411186e-02
39461,rs73439311_RC,rs73439311,neg,ref,225.047471,518.727303,1.021873,15.889120,11.292765,264.268231,...,5.915880,1.319524,-0.768413,-0.555736,-0.688659,2.142339,1.440353,0.851587,5.096070e-12,4.791545e-02
39467,rs2234058,rs2234058,pos,ref,673.043758,1373.481543,0.938229,34.201737,29.605382,587.687732,...,37.364005,32.767650,0.089103,0.123557,0.102023,0.925545,0.622003,1.145240,2.480950e-30,1.707459e-33


Alternatively:

In [38]:
# Select rows based on threshold on the pvalue of 0.01 
lcl_pvalue_filter = lcl_analysis[(lcl_analysis['ref_pvalue'] <= 0.01) |(lcl_analysis['alt_pvalue'] <= 0.01)]
lcl_pvalue_filter

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,alt_logp,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue
0,rs11548103_RC,rs11548103,neg,ref,893.147913,1403.234147,0.637129,20.726159,16.129804,985.132571,...,24.239298,19.642943,-0.157649,-0.070399,-0.124930,1.197350,0.803895,0.906627,7.416455e-17,2.275395e-20
3,rs11102212_RC,rs11102212,neg,ref,272.682393,724.187989,1.276380,20.903079,16.306723,270.606119,...,18.276201,13.679846,-0.182603,0.057417,-0.092596,0.151356,0.102546,1.007673,4.934880e-17,2.090037e-14
6,rs112338151,rs112338151,pos,ref,311.022339,938.600426,1.454065,35.570382,30.974027,441.094084,...,46.863102,42.266747,-0.149852,-0.070538,-0.120109,1.108686,0.743469,0.705116,1.061629e-31,5.410694e-43
14,rs10910099_RC,rs10910099,neg,ref,1565.369298,2410.813917,0.609195,18.726474,14.130119,1597.275307,...,4.277202,0.000000,-0.319983,-0.442277,-0.365843,2.609976,1.744575,0.980025,7.411069e-15,1.000000e+00
17,rs61731104,rs61731104,pos,ref,787.310108,1106.543203,0.472118,7.158719,2.562364,851.116339,...,2.071566,0.000000,-0.426533,-0.181428,-0.334618,1.448329,0.973007,0.925032,2.739278e-03,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39435,rs6002380_RC,rs6002380,neg,ref,1021.939757,1393.401924,0.436551,7.957672,3.361317,1410.403292,...,3.737057,0.000000,-0.072219,-0.194436,-0.118050,0.225938,0.152430,0.724573,4.351945e-04,1.000000e+00
39454,rs12165508_RC,rs12165508,neg,ref,905.677775,1186.698469,0.370023,10.844473,6.248118,602.818906,...,5.622711,1.026356,-0.004909,0.002897,-0.001982,0.082316,0.061067,1.502404,5.647837e-07,9.411186e-02
39461,rs73439311_RC,rs73439311,neg,ref,225.047471,518.727303,1.021873,15.889120,11.292765,264.268231,...,5.915880,1.319524,-0.768413,-0.555736,-0.688659,2.142339,1.440353,0.851587,5.096070e-12,4.791545e-02
39467,rs2234058,rs2234058,pos,ref,673.043758,1373.481543,0.938229,34.201737,29.605382,587.687732,...,37.364005,32.767650,0.089103,0.123557,0.102023,0.925545,0.622003,1.145240,2.480950e-30,1.707459e-33


In [39]:
#Check if there is overlap in snps between the two dataframes 
#if list is empty then there is complete overlap (rows in the dataframes are identical)
list(set(df_tewhey.snp) - set(lcl_pvalue_filter.snp))

[]

filtering on pvalue for A and B give the same snp's as dropping the NaN values


In [40]:
#Check for 0 values per column
df_tewhey.isin([0]).sum()

id                 0
snp                0
direction          0
haplotype          0
ref_ctrl_mean      0
ref_exp_mean       0
ref_log2fc         0
ref_logp           0
ref_logpadj      662
alt_ctrl_mean      0
alt_exp_mean       0
alt_log2fc         0
alt_logp           0
alt_logpadj      671
logskew_12878      0
logskew_19239      0
logskew_comb       0
c_skew_logp        4
c_skew_fdr         4
beta               0
ref_pvalue         0
alt_pvalue         0
dtype: int64

In [41]:
# Remove rows with zero value to see how much unique variants would be left
non_zero = df_tewhey.loc[(df_tewhey['ref_logpadj'] != 0) & (df_tewhey['alt_logpadj'] != 0) & (df_tewhey['c_skew_fdr'] != 0)]
f'amount of unique variants left after removing zero values: {len(non_zero.snp.value_counts())}'

'amount of unique variants left after removing zero values: 2464'

Way to less variants compared to 3435 mentioned in the paper.

***

Check contents snp column

In [42]:
df_tewhey.head()

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,alt_logp,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue
0,rs11548103_RC,rs11548103,neg,ref,893.147913,1403.234147,0.637129,20.726159,16.129804,985.132571,...,24.239298,19.642943,-0.157649,-0.070399,-0.124930,1.197350,0.803895,0.906627,7.416455e-17,2.275395e-20
3,rs11102212_RC,rs11102212,neg,ref,272.682393,724.187989,1.276380,20.903079,16.306723,270.606119,...,18.276201,13.679846,-0.182603,0.057417,-0.092596,0.151356,0.102546,1.007673,4.934880e-17,2.090037e-14
6,rs112338151,rs112338151,pos,ref,311.022339,938.600426,1.454065,35.570382,30.974027,441.094084,...,46.863102,42.266747,-0.149852,-0.070538,-0.120109,1.108686,0.743469,0.705116,1.061629e-31,5.410694e-43
14,rs10910099_RC,rs10910099,neg,ref,1565.369298,2410.813917,0.609195,18.726474,14.130119,1597.275307,...,4.277202,0.000000,-0.319983,-0.442277,-0.365843,2.609976,1.744575,0.980025,7.411069e-15,1.000000e+00
17,rs61731104,rs61731104,pos,ref,787.310108,1106.543203,0.472118,7.158719,2.562364,851.116339,...,2.071566,0.000000,-0.426533,-0.181428,-0.334618,1.448329,0.973007,0.925032,2.739278e-03,1.000000e+00


In [43]:
df_tewhey['snp'].value_counts()

rs116961551         4
rs117646503         4
rs112767110         4
rs117676123         4
rs115582002         4
                   ..
rs1047207           1
rs56089143          1
rs10081322          1
chr7:127142915:I    1
rs2510053           1
Name: snp, Length: 3595, dtype: int64

In [44]:
rs_ids = [i for i in df_tewhey['snp'] if i.startswith('rs')]
f'{len(rs_ids)} variants with a rs id'

'4075 variants with a rs id'

In [45]:
unique_rs_id = set(rs_ids)
f'amount of variants with an unique rs id: {len(unique_rs_id)}'

'amount of variants with an unique rs id: 3377'

In [46]:
f'Variants without an unique rs id: {len(df_tewhey.snp.value_counts()) - len(unique_rs_id)}'

'Variants without an unique rs id: 218'

In [47]:
# Inspect the variants that do not have and rs id
snps_without_rsid = [i for i in df_tewhey['snp'].unique() if not i.startswith('rs')]
#Check content: first 5 elements of the list
snps_without_rsid[0:5]

['chr1:150824527:I',
 'chr1:205753876:D',
 'chr1:171993854:D',
 'chr1:41499967:I',
 'chr1:22352395:D']

In [48]:
f'rows that do not have an unique rs id: {len(snps_without_rsid)}'

'rows that do not have an unique rs id: 218'

In [49]:
snps_without_pos = [i for i in snps_without_rsid if not i.startswith('chr')]
snps_without_pos

['MERGED_DEL_2_8411', 'MERGED_DEL_2_98507']

Check for the duplicate entries without rs in dataset

In [50]:
df_tewhey.snp.value_counts()

rs116961551         4
rs117646503         4
rs112767110         4
rs117676123         4
rs115582002         4
                   ..
rs1047207           1
rs56089143          1
rs10081322          1
chr7:127142915:I    1
rs2510053           1
Name: snp, Length: 3595, dtype: int64

In [51]:
#Check snp that's in there multiple times
df_tewhey.loc[df_tewhey.snp == 'rs116961551']
#difference is in haplotype and direction - surrounding sequence: ref=reference, alt=alternative allele for neighboring variants

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,alt_logp,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue
34490,rs116961551,rs116961551,pos,ref,287.807720,564.053005,0.890408,11.500903,6.904547,397.119740,...,29.791420,25.195065,0.439450,0.330744,0.398685,2.405575,1.617816,0.724738,1.245812e-07,6.381683e-26
34491,rs116961551_RC,rs116961551,neg,ref,371.105592,1411.171801,1.846791,90.733295,86.136940,258.103539,...,71.166277,66.569921,0.153470,0.176314,0.162037,0.697980,0.469570,1.437817,7.295579e-87,2.692022e-67
34492,rs116961551_alt,rs116961551,pos,alt,248.249497,568.865984,1.064622,13.544769,8.948414,435.085332,...,33.276889,28.680533,0.325865,0.223755,0.287574,0.611513,0.407974,0.570577,1.126123e-09,2.086732e-29
34493,rs116961551_RC_alt,rs116961551,neg,alt,431.555150,1430.754510,1.628778,56.057720,51.461364,350.173018,...,94.525782,89.929427,0.513634,0.544316,0.525140,2.089192,1.405963,1.232405,3.456492e-52,1.176449e-90


***

Check if ther is overlap in position (nr) for snps/variants containing :D and :I

In [52]:
#Select entries ending on D
deletion = [i for i in snps_without_rsid if i.endswith('D')]
#Select entries ending on I
insertion = [i for i in snps_without_rsid if i.endswith('I')]
#snp's that do not start with chr
merged = [ i for i in snps_without_rsid if not i.startswith('chr')]

In [53]:
#remove last two characters of elements in lists
del_snp = [i[:-2] for i in deletion]
ins_snp = [i[:-2] for i in insertion]

#Check if there are overlapping snps in both lists
intersect = list(set(del_snp) & set(ins_snp))
f'The overlapping snps are: {intersect}'

'The overlapping snps are: []'

Check entries left with no rs or chr position

In [54]:
#Check contents of variants without rsid or position
merged

['MERGED_DEL_2_8411', 'MERGED_DEL_2_98507']

In [55]:
df_tewhey.loc[(df_tewhey['snp'] == 'MERGED_DEL_2_8411')|  (df_tewhey['snp'] =='MERGED_DEL_2_98507')]
#Duplicate snp differs in Direction: of oligo relative to TSS

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,alt_logp,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue
960,MERGED_DEL_2_8411,MERGED_DEL_2_8411,pos,ref,796.731749,1053.169443,0.384989,3.815508,0.000000,873.070571,...,8.259494,3.663139,0.139892,0.155051,0.145576,0.601242,0.401369,0.912563,1.000000e+00,2.172008e-04
35058,MERGED_DEL_2_98507,MERGED_DEL_2_98507,pos,ref,1043.167585,1414.947703,0.406308,6.636056,2.039701,1481.067018,...,19.868321,15.271966,0.177758,0.572640,0.325839,1.321323,0.889750,0.704335,9.126399e-03,5.346063e-16
35059,MERGED_DEL_2_98507_RC,MERGED_DEL_2_98507,neg,ref,1330.610486,2260.741079,0.718237,21.712406,17.116051,1491.772628,...,29.960524,25.364169,-0.018444,0.251394,0.082745,0.877501,0.588817,0.891966,7.655073e-18,4.323460e-26


***

### From grCh37 to grCh38

In [56]:
# Create dataframe of snps without an rs id
df_chr = df_tewhey.loc[df_tewhey.snp.isin(snps_without_rsid)]
df_chr

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,alt_logp,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue
34,chr1:150824527:I_RC,chr1:150824527:I,neg,ref,665.673312,928.793886,0.467121,7.948176,3.351821,616.995944,...,5.977152,1.380797,-0.149947,0.169805,-0.030040,0.385947,0.256515,1.078894,4.448146e-04,4.161048e-02
331,chr1:205753876:D_RC,chr1:205753876:D,neg,ref,1420.436964,3614.404655,1.317902,87.999640,83.403285,888.001576,...,37.710249,33.113894,-0.365522,-0.288343,-0.336580,2.764688,1.837817,1.599588,3.951077e-84,7.693188e-34
402,chr1:171993854:D,chr1:171993854:D,pos,ref,1274.639199,1766.597922,0.437787,11.528658,6.932303,1447.944789,...,10.483445,5.887090,0.237896,-0.094620,0.113202,0.510838,0.340568,0.880309,1.168683e-07,1.296911e-06
700,chr1:41499967:I,chr1:41499967:I,pos,ref,978.055615,1954.770916,0.954079,62.441074,57.844719,1054.557178,...,18.463807,13.867452,-0.207228,-0.232217,-0.216599,0.715108,0.480724,0.927456,1.429820e-58,1.356900e-14
717,chr1:22352395:D,chr1:22352395:D,pos,ref,1328.979443,9369.413644,2.676462,283.389681,278.793326,3635.588956,...,277.421672,272.825317,-0.016826,-0.086135,-0.042817,0.487141,0.322816,0.365547,1.609437e-279,1.495144e-273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38176,chr21:38345364:I_RC,chr21:38345364:I,neg,ref,901.072839,1388.354049,0.599901,12.677858,8.081503,890.896221,...,3.770434,0.000000,-0.314867,-0.166490,-0.259226,0.871231,0.584102,1.011423,8.288911e-09,1.000000e+00
38177,chr21:38345364:I_RC_alt,chr21:38345364:I,neg,alt,743.009986,1093.456565,0.496592,2.826263,0.000000,712.066055,...,10.387111,5.790755,0.063836,0.002508,0.040838,0.020828,0.017005,1.043457,1.000000e+00,1.618991e-06
38182,chr21:30327732:D_RC,chr21:30327732:D,neg,ref,370.915260,841.553676,1.114396,23.234202,18.637847,589.439600,...,40.161885,35.565530,-0.057799,0.264200,0.062951,0.506545,0.337016,0.629268,2.302252e-19,2.719383e-36
38704,chr22:50310881:D_alt,chr22:50310881:D,pos,alt,1355.235110,1261.597715,-0.101515,0.473329,0.000000,1375.781057,...,7.138922,2.542566,-0.294019,-0.157692,-0.242897,2.150959,1.447458,0.985066,1.000000e+00,2.867039e-03


In [57]:
#Remove rows that we can't find positions for
df_chr = df_chr[(df_chr['snp'] != "MERGED_DEL_2_8411") & (df_chr['snp'] != "MERGED_DEL_2_98507")]

In [58]:
# Input assembly converter ensmbl: chromosome, start position, stop position
# Add an stop position just in case the position in removed in the new build
snp = list(df_chr['snp'].unique())
start_pos = [int(i[5:].replace(':', '').replace('I','').replace('D', '')) for i in snp if i.startswith('chr')]
end_pos = [i + 1 for i in start_pos]
chr_pos = [i[:5].replace(':', '') for i in snp if i.startswith('chr')]

In [59]:
len(snp)

216

In [60]:
# Create a txt file in bed format for input in assembly converter of ensmbl
with open("data\grch37_lcl_sig_input_assembly_converter.txt","w") as f:
    for (chr_pos,start_pos,end_pos, snps) in zip(chr_pos,start_pos,end_pos,snp):
        f.write("{0} \t {1} \t {2} \t {3}\n".format(chr_pos,start_pos,end_pos, snps))

In [61]:
len(snp)

216

In [62]:
# Load output file assembly converter
bedfile = pd.read_csv('8vqpcqjVo8VYHeel.bed', sep='\t', header=None)

In [63]:
bedfile

,0,1,2
0,chr1,150852051,150852052
1,chr1,205784748,205784749
2,chr1,172024714,172024715
3,chr1,41034295,41034296
4,chr1,22025902,22025903
...,...,...,...
252,chr21,36973064,36973065
253,chr21,36973064,36973065
254,chr21,28955410,28955411
255,chr22,49917233,49917234


In [64]:
if len(bedfile) == len(df_chr['snp']):
    print('True')
else:
    print('False')

True


In [65]:
lcl_sig = pd.read_csv('data/significant_lcl_38.bed', sep='\t', header=None)

In [66]:
lcl_sig

,0,1,2,3
0,chr19,48875442,48875443,chr19:49378699:D
1,chr1,22025902,22025903,chr1:22352395:D
2,chr6,26473868,26473869,chr6:26474096:I
3,chr8,70748873,70748874,chr8:71661108:I
4,chr17,46059643,46059644,chr17:44137009:D
...,...,...,...,...
208,chr6,29857858,29857859,chr6:29825635:I
209,chr17,45939235,45939236,chr17:44016601:I
210,chr2,206765097,206765098,chr2:207629821:I
211,chr5,79967338,79967339,chr5:79263161:I


In [67]:
if len(lcl_sig[3]) == len(df_chr['snp'].unique()):
    print('True')
else:
    print(f'False. There are {len(df_chr.snp.unique())- len(lcl_sig[3])} positions missing')

False. There are 3 positions missing


In [68]:
#Find the missing snps
df_chr[~df_chr['snp'].isin(list(lcl_sig[3]))]

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,alt_logp,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue
15374,chr10:51583018:D_RC,chr10:51583018:D,neg,ref,182.744215,453.229658,1.186026,16.662832,12.066477,256.769640,...,0.678229,0.000000,-0.950120,-0.843468,-0.910125,1.952377,1.313647,0.711705,8.580703e-13,1.000000e+00
29079,chr17:36904739:D,chr17:36904739:D,pos,ref,1594.207418,2515.344218,0.615043,22.826099,18.229744,1589.629540,...,54.936248,50.339893,0.409717,0.474387,0.433968,4.923863,3.157711,1.002880,5.891915e-19,4.572007e-51
29080,chr17:36904739:D_RC,chr17:36904739:D,neg,ref,1546.975363,3675.040906,1.170987,69.106165,64.509810,957.421529,...,93.211848,88.615492,0.152803,0.324841,0.217317,1.512971,1.016804,1.615772,3.091646e-65,2.423860e-89
29411,chr17:36438743:I_RC,chr17:36438743:I,neg,ref,1110.254661,1702.627803,0.598501,10.226223,5.629867,888.939888,...,60.284617,55.688262,0.788642,0.684250,0.749495,4.144482,2.724976,1.248965,2.344945e-06,2.049925e-56


Manual check: <br>
chr 10:1, chr 17:2 missing <br>
- chr10   51583018    51583019
- chr17   36904739    36904740
- chr17   36438743    36438744 <br>
<br>
Rerun with just these positions works, added to file
- chr10	51253024	51253025
- chr17	34158265	34158266
- chr17	33692267	33692268 <br>

## Positions can't be found a second time!

##

In [69]:
#Load complete file converted positions to build 38
#grch38 = pd.read_csv('grch38_chromosome_positions.bed',sep='\t', header=None, names=['snp_chromosome', 'snp_position', 'pos+1'])

In [70]:
# View grch38 file
#grch38

In [71]:
# #Check if the file has the same length as the dataframe
# if len(grch38) == len(df_chr['snp']):
#     print('True')
# else:
#     print('False')

In [72]:
#grch38.snp_chromosome = grch38.snp_chromosome#.str[3:]

In [73]:
# grch38.insert(0,'snp', list(df_chr['snp']))
# grch38 = grch38.iloc[:,:-1].drop_duplicates()

In [74]:
# grch38['snp'].value_counts()

In [75]:
# grch38[grch38['snp'] == 'chr17:43775212:D']

In [76]:
# grch38[grch38['snp'] == 'chr17:36904739:D']

In [77]:
# df_tewhey[df_tewhey['snp'] == 'chr17:43775212:D']

***

"a corrected p-value of 0.01 or less in either the reference or alternate allele in order to call a sequence as having a
regulatory effect on expression." <br>

In [78]:
#create column non log p values
dfbeta['skew_p'] = dfbeta['c_skew_logp'].map(lambda x: 10 ** -x)
dfbeta['fdr'] = dfbeta['c_skew_fdr'].map(lambda x: 10 ** -x)


In [79]:
dfbeta.sort_values(by='c_skew_fdr', ascending=False)

,id,snp,ref_ctrl_mean,alt_ctrl_mean,c_skew_logp,c_skew_fdr,beta,skew_p,fdr
3988,rs1617806,rs1617806,1249.963384,1223.635583,8.958733,5.321744,1.021516,1.099681e-09,0.000005
30970,rs11080327,rs11080327,1374.536239,918.581377,8.557923,5.221964,1.496369,2.767432e-09,0.000006
11605,rs2191501,rs2191501,1143.306328,1281.564244,7.402123,4.393105,0.892118,3.961654e-08,0.000040
8432,rs2351011,rs2351011,1056.011325,1161.644047,7.331125,4.393105,0.909066,4.665256e-08,0.000040
2271,rs9661285,rs9661285,1901.184464,1113.711962,7.422196,4.393105,1.707070,3.782722e-08,0.000040
...,...,...,...,...,...,...,...,...,...
19946,rs114196109_RC,rs114196109,353.834384,338.525982,0.001750,0.001350,1.045221,9.959775e-01,0.996897
34290,rs76154201_alt,rs76154201,8.356027,16.878985,0.000000,0.000000,0.495055,1.000000e+00,1.000000
25835,rs2136751_RC,rs2136751,1.735254,20.676853,0.000000,0.000000,0.083923,1.000000e+00,1.000000
32918,rs75530705_alt,rs75530705,336.399370,9.215217,0.000000,0.000000,36.504769,1.000000e+00,1.000000


In [80]:
len(dfbeta.loc[dfbeta['fdr'] <= 0.05].snp.value_counts())

924

In [81]:
len(dfbeta.loc[dfbeta['skew_p'] <= 0.01].snp.value_counts())

870

In [82]:
10**-1

0.1

***

In [83]:
df_tewhey

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,alt_logp,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue
0,rs11548103_RC,rs11548103,neg,ref,893.147913,1403.234147,0.637129,20.726159,16.129804,985.132571,...,24.239298,19.642943,-0.157649,-0.070399,-0.124930,1.197350,0.803895,0.906627,7.416455e-17,2.275395e-20
3,rs11102212_RC,rs11102212,neg,ref,272.682393,724.187989,1.276380,20.903079,16.306723,270.606119,...,18.276201,13.679846,-0.182603,0.057417,-0.092596,0.151356,0.102546,1.007673,4.934880e-17,2.090037e-14
6,rs112338151,rs112338151,pos,ref,311.022339,938.600426,1.454065,35.570382,30.974027,441.094084,...,46.863102,42.266747,-0.149852,-0.070538,-0.120109,1.108686,0.743469,0.705116,1.061629e-31,5.410694e-43
14,rs10910099_RC,rs10910099,neg,ref,1565.369298,2410.813917,0.609195,18.726474,14.130119,1597.275307,...,4.277202,0.000000,-0.319983,-0.442277,-0.365843,2.609976,1.744575,0.980025,7.411069e-15,1.000000e+00
17,rs61731104,rs61731104,pos,ref,787.310108,1106.543203,0.472118,7.158719,2.562364,851.116339,...,2.071566,0.000000,-0.426533,-0.181428,-0.334618,1.448329,0.973007,0.925032,2.739278e-03,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39435,rs6002380_RC,rs6002380,neg,ref,1021.939757,1393.401924,0.436551,7.957672,3.361317,1410.403292,...,3.737057,0.000000,-0.072219,-0.194436,-0.118050,0.225938,0.152430,0.724573,4.351945e-04,1.000000e+00
39454,rs12165508_RC,rs12165508,neg,ref,905.677775,1186.698469,0.370023,10.844473,6.248118,602.818906,...,5.622711,1.026356,-0.004909,0.002897,-0.001982,0.082316,0.061067,1.502404,5.647837e-07,9.411186e-02
39461,rs73439311_RC,rs73439311,neg,ref,225.047471,518.727303,1.021873,15.889120,11.292765,264.268231,...,5.915880,1.319524,-0.768413,-0.555736,-0.688659,2.142339,1.440353,0.851587,5.096070e-12,4.791545e-02
39467,rs2234058,rs2234058,pos,ref,673.043758,1373.481543,0.938229,34.201737,29.605382,587.687732,...,37.364005,32.767650,0.089103,0.123557,0.102023,0.925545,0.622003,1.145240,2.480950e-30,1.707459e-33


In [84]:
#df_tewhey.sort_values(by='ref_pvalue', ascending=False)

In [85]:
df_tewhey['fdr'] = df_tewhey['c_skew_fdr'].map(lambda x: 10 ** -x)
fdr = df_tewhey[df_tewhey['fdr'] <= 0.05]

In [86]:
fdr

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue,fdr
14,rs10910099_RC,rs10910099,neg,ref,1565.369298,2410.813917,0.609195,18.726474,14.130119,1597.275307,...,0.000000,-0.319983,-0.442277,-0.365843,2.609976,1.744575,0.980025,7.411069e-15,1.000000e+00,0.018006
56,rs56178029,rs56178029,pos,ref,1085.782838,1684.427500,0.624392,29.144073,24.547717,1456.292347,...,132.618977,0.914089,1.055694,0.967191,5.608785,3.551579,0.745580,2.833235e-25,2.404488e-133,0.000281
144,rs35883594,rs35883594,pos,ref,972.834942,4072.735029,2.019667,192.748518,188.152163,1078.011867,...,18.912175,-1.264322,-1.393424,-1.312736,5.092412,3.264804,0.902434,7.044288e-189,1.224123e-19,0.000543
186,rs2457823,rs2457823,pos,ref,1271.913478,2921.442764,1.130604,79.154270,74.557915,860.513778,...,127.459518,0.567486,0.717002,0.623554,3.421907,2.257675,1.478086,2.767486e-75,3.471220e-128,0.005525
230,rs12044559_alt,rs12044559,pos,alt,711.484617,1241.074038,0.731404,24.150482,19.554127,644.681018,...,48.814916,0.351085,0.130786,0.268473,2.864907,1.906437,1.103623,2.791728e-20,1.531384e-49,0.012404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39126,rs7293064_RC_alt,rs7293064,neg,alt,1199.390610,2625.811458,1.099562,41.503023,36.906668,1176.475150,...,16.085631,-0.348721,-0.231136,-0.304627,2.106100,1.414579,1.019478,1.239743e-37,8.210482e-17,0.038496
39170,rs9610708_RC,rs9610708,neg,ref,1130.884118,2007.284444,0.721886,33.110367,28.514011,1103.037063,...,6.533126,-0.337247,-0.382478,-0.354209,3.143543,2.078998,1.025246,3.061883e-29,2.930044e-07,0.008337
39316,rs6002851_RC,rs6002851,neg,ref,840.115436,1406.356821,0.709199,21.996948,17.400593,736.108567,...,0.569784,-0.290304,-0.300597,-0.294164,2.130048,1.432578,1.141293,3.975642e-18,2.692876e-01,0.036934
39411,rs2272804,rs2272804,pos,ref,1249.354490,1633.960228,0.371340,9.337950,4.741594,1245.604249,...,0.000000,-0.377239,-0.561062,-0.446172,2.623654,1.754817,1.003011,1.813033e-05,1.000000e+00,0.017587


***
Add column to see which variants are differentially expressed within the dataframe

In [87]:
id = list(fdr.id)

In [88]:
df_tewhey['differentially_expressed'] = np.where(df_tewhey['id'].isin(id), 'yes', 'no')

In [89]:
df_tewhey[df_tewhey['differentially_expressed'] == 'yes']

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue,fdr,differentially_expressed
14,rs10910099_RC,rs10910099,neg,ref,1565.369298,2410.813917,0.609195,18.726474,14.130119,1597.275307,...,-0.319983,-0.442277,-0.365843,2.609976,1.744575,0.980025,7.411069e-15,1.000000e+00,0.018006,yes
56,rs56178029,rs56178029,pos,ref,1085.782838,1684.427500,0.624392,29.144073,24.547717,1456.292347,...,0.914089,1.055694,0.967191,5.608785,3.551579,0.745580,2.833235e-25,2.404488e-133,0.000281,yes
144,rs35883594,rs35883594,pos,ref,972.834942,4072.735029,2.019667,192.748518,188.152163,1078.011867,...,-1.264322,-1.393424,-1.312736,5.092412,3.264804,0.902434,7.044288e-189,1.224123e-19,0.000543,yes
186,rs2457823,rs2457823,pos,ref,1271.913478,2921.442764,1.130604,79.154270,74.557915,860.513778,...,0.567486,0.717002,0.623554,3.421907,2.257675,1.478086,2.767486e-75,3.471220e-128,0.005525,yes
230,rs12044559_alt,rs12044559,pos,alt,711.484617,1241.074038,0.731404,24.150482,19.554127,644.681018,...,0.351085,0.130786,0.268473,2.864907,1.906437,1.103623,2.791728e-20,1.531384e-49,0.012404,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39126,rs7293064_RC_alt,rs7293064,neg,alt,1199.390610,2625.811458,1.099562,41.503023,36.906668,1176.475150,...,-0.348721,-0.231136,-0.304627,2.106100,1.414579,1.019478,1.239743e-37,8.210482e-17,0.038496,yes
39170,rs9610708_RC,rs9610708,neg,ref,1130.884118,2007.284444,0.721886,33.110367,28.514011,1103.037063,...,-0.337247,-0.382478,-0.354209,3.143543,2.078998,1.025246,3.061883e-29,2.930044e-07,0.008337,yes
39316,rs6002851_RC,rs6002851,neg,ref,840.115436,1406.356821,0.709199,21.996948,17.400593,736.108567,...,-0.290304,-0.300597,-0.294164,2.130048,1.432578,1.141293,3.975642e-18,2.692876e-01,0.036934,yes
39411,rs2272804,rs2272804,pos,ref,1249.354490,1633.960228,0.371340,9.337950,4.741594,1245.604249,...,-0.377239,-0.561062,-0.446172,2.623654,1.754817,1.003011,1.813033e-05,1.000000e+00,0.017587,yes


In [90]:
fdr[(fdr['ref_pvalue'] <= 0.01) | (fdr['alt_pvalue'] <= 0.01)]#.snp.value_counts()

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue,fdr
14,rs10910099_RC,rs10910099,neg,ref,1565.369298,2410.813917,0.609195,18.726474,14.130119,1597.275307,...,0.000000,-0.319983,-0.442277,-0.365843,2.609976,1.744575,0.980025,7.411069e-15,1.000000e+00,0.018006
56,rs56178029,rs56178029,pos,ref,1085.782838,1684.427500,0.624392,29.144073,24.547717,1456.292347,...,132.618977,0.914089,1.055694,0.967191,5.608785,3.551579,0.745580,2.833235e-25,2.404488e-133,0.000281
144,rs35883594,rs35883594,pos,ref,972.834942,4072.735029,2.019667,192.748518,188.152163,1078.011867,...,18.912175,-1.264322,-1.393424,-1.312736,5.092412,3.264804,0.902434,7.044288e-189,1.224123e-19,0.000543
186,rs2457823,rs2457823,pos,ref,1271.913478,2921.442764,1.130604,79.154270,74.557915,860.513778,...,127.459518,0.567486,0.717002,0.623554,3.421907,2.257675,1.478086,2.767486e-75,3.471220e-128,0.005525
230,rs12044559_alt,rs12044559,pos,alt,711.484617,1241.074038,0.731404,24.150482,19.554127,644.681018,...,48.814916,0.351085,0.130786,0.268473,2.864907,1.906437,1.103623,2.791728e-20,1.531384e-49,0.012404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39126,rs7293064_RC_alt,rs7293064,neg,alt,1199.390610,2625.811458,1.099562,41.503023,36.906668,1176.475150,...,16.085631,-0.348721,-0.231136,-0.304627,2.106100,1.414579,1.019478,1.239743e-37,8.210482e-17,0.038496
39170,rs9610708_RC,rs9610708,neg,ref,1130.884118,2007.284444,0.721886,33.110367,28.514011,1103.037063,...,6.533126,-0.337247,-0.382478,-0.354209,3.143543,2.078998,1.025246,3.061883e-29,2.930044e-07,0.008337
39316,rs6002851_RC,rs6002851,neg,ref,840.115436,1406.356821,0.709199,21.996948,17.400593,736.108567,...,0.569784,-0.290304,-0.300597,-0.294164,2.130048,1.432578,1.141293,3.975642e-18,2.692876e-01,0.036934
39411,rs2272804,rs2272804,pos,ref,1249.354490,1633.960228,0.371340,9.337950,4.741594,1245.604249,...,0.000000,-0.377239,-0.561062,-0.446172,2.623654,1.754817,1.003011,1.813033e-05,1.000000e+00,0.017587


In [91]:
print('variants active for ref AND alt with fdr < 0.05:',len(fdr[(fdr['ref_pvalue'] <= 0.01) & (fdr['alt_pvalue'] <= 0.01)].snp.value_counts()))

variants active for ref AND alt with fdr < 0.05: 512


In [92]:
print('variants significant (<0.01) for ref AND alt total variants:',len(df_tewhey[(df_tewhey['ref_pvalue'] <= 0.01) | (df_tewhey['alt_pvalue'] <= 0.01)].snp.value_counts()))

variants significant (<0.01) for ref AND alt total variants: 3595


In [93]:
# fdr values might be in log values as well, reverse:
dfbeta['fdr'] = dfbeta['c_skew_fdr'].map(lambda x: 10 ** -x)
f'{len((dfbeta[dfbeta.fdr <= 0.05]).snp.value_counts())} unique variants left with fdr <= 0.05'

'924 unique variants left with fdr <= 0.05'

***

#### reproduce attempt log skew comb column

In [94]:
logskew = lcl_analysis[['id','snp', 'logskew_12878', 'logskew_19239', 'logskew_comb']]

In [95]:
logskew

,id,snp,logskew_12878,logskew_19239,logskew_comb
0,rs11548103_RC,rs11548103,-0.157649,-0.070399,-0.124930
1,rs2016366,rs2016366,0.344054,0.696614,0.476264
2,rs2016366_alt,rs2016366,0.390497,0.395150,0.392242
3,rs11102212_RC,rs11102212,-0.182603,0.057417,-0.092596
4,rs646867_RC,rs646867,-0.287121,-0.179513,-0.246768
...,...,...,...,...,...
39473,rs9621715_alt,rs9621715,-0.089697,-0.008966,-0.059423
39474,rs4275_RC,rs4275,0.167059,0.269337,0.205413
39475,rs131816_RC,rs131816,0.473005,0.270571,0.397092
39476,rs131816_RC_alt,rs131816,-0.011147,-0.019440,-0.014257


In [96]:
#logskew['avg_logskew'] = logskew.loc[:,['logskew_12878', 'logskew_19239']].sum(axis=1)

In [97]:
logskew['avg_logskew'] = (logskew['logskew_12878'] + logskew['logskew_19239']).divide(2)

In [98]:
logskew

,id,snp,logskew_12878,logskew_19239,logskew_comb,avg_logskew
0,rs11548103_RC,rs11548103,-0.157649,-0.070399,-0.124930,-0.114024
1,rs2016366,rs2016366,0.344054,0.696614,0.476264,0.520334
2,rs2016366_alt,rs2016366,0.390497,0.395150,0.392242,0.392824
3,rs11102212_RC,rs11102212,-0.182603,0.057417,-0.092596,-0.062593
4,rs646867_RC,rs646867,-0.287121,-0.179513,-0.246768,-0.233317
...,...,...,...,...,...,...
39473,rs9621715_alt,rs9621715,-0.089697,-0.008966,-0.059423,-0.049331
39474,rs4275_RC,rs4275,0.167059,0.269337,0.205413,0.218198
39475,rs131816_RC,rs131816,0.473005,0.270571,0.397092,0.371788
39476,rs131816_RC_alt,rs131816,-0.011147,-0.019440,-0.014257,-0.015293


Note to self: Figure out how logskew_comb is created <br>
supplementary paper?

***

FDR.

In [99]:
lcl_analysis

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,alt_logp,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue
0,rs11548103_RC,rs11548103,neg,ref,893.147913,1403.234147,0.637129,20.726159,16.129804,985.132571,...,24.239298,19.642943,-0.157649,-0.070399,-0.124930,1.197350,0.803895,0.906627,7.416455e-17,2.275395e-20
1,rs2016366,rs2016366,pos,ref,316.596386,258.902025,-0.281146,1.799521,0.000000,345.506770,...,0.437249,0.000000,0.344054,0.696614,0.476264,NaN,NaN,0.916325,1.000000e+00,1.000000e+00
2,rs2016366_alt,rs2016366,pos,alt,653.148636,605.357051,-0.104744,0.364373,0.000000,627.652912,...,1.736553,0.000000,0.390497,0.395150,0.392242,NaN,NaN,1.040621,1.000000e+00,1.000000e+00
3,rs11102212_RC,rs11102212,neg,ref,272.682393,724.187989,1.276380,20.903079,16.306723,270.606119,...,18.276201,13.679846,-0.182603,0.057417,-0.092596,0.151356,0.102546,1.007673,4.934880e-17,2.090037e-14
4,rs646867_RC,rs646867,neg,ref,605.412960,595.896429,-0.023234,0.213685,0.000000,978.751908,...,2.243712,0.000000,-0.287121,-0.179513,-0.246768,NaN,NaN,0.618556,1.000000e+00,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39473,rs9621715_alt,rs9621715,pos,alt,650.529931,507.281685,-0.342574,2.658698,0.000000,563.523072,...,2.811763,0.000000,-0.089697,-0.008966,-0.059423,NaN,NaN,1.154398,1.000000e+00,1.000000e+00
39474,rs4275_RC,rs4275,neg,ref,718.716545,859.461801,0.250261,2.474741,0.000000,576.804932,...,5.422339,0.825984,0.167059,0.269337,0.205413,NaN,NaN,1.246031,1.000000e+00,1.492850e-01
39475,rs131816_RC,rs131816,neg,ref,754.841862,634.680055,-0.245811,1.350291,0.000000,954.736488,...,1.036517,0.000000,0.473005,0.270571,0.397092,NaN,NaN,0.790628,1.000000e+00,1.000000e+00
39476,rs131816_RC_alt,rs131816,neg,alt,827.703506,840.290995,0.018192,0.117288,0.000000,753.191408,...,0.110077,0.000000,-0.011147,-0.019440,-0.014257,NaN,NaN,1.098929,1.000000e+00,1.000000e+00


In [100]:
lcl_analysis['c_skew_logp'].isna().sum()

35143

In [101]:
#Change log p to just p values for stats.false_discovery_control() to work
logp = lcl_analysis['c_skew_logp'].dropna()
logp = logp.loc[logp != 0]
logp = logp.tolist()

In [102]:
listp = []
for i in logp:
    d =10**-i
    listp.append(d)
p = np.array(listp)

In [103]:
from scipy import stats

In [104]:
#stats.false_discovery_control(p)

In [105]:
fdr = stats.false_discovery_control(p, method='bh')
fdr

array([0.1569293 , 0.78895604, 0.18035586, ..., 0.03624481, 0.23855927,
       0.45221684])

In [106]:
len(fdr)

4331

In [107]:
df3 = lcl_analysis[['snp','c_skew_logp', 'c_skew_fdr']].dropna()
df3 = df3.loc[(df3['c_skew_fdr'] != 0) & df3['c_skew_logp'] != 0]

In [108]:
df3['skew_p'] = p
df3['stats_fdr'] = fdr

In [109]:
df3

,snp,c_skew_logp,c_skew_fdr,skew_p,stats_fdr
0,rs11548103,1.197350,0.803895,0.063482,0.156929
3,rs11102212,0.151356,0.102546,0.705738,0.788956
6,rs112338151,1.108686,0.743469,0.077860,0.180356
14,rs10910099,2.609976,1.744575,0.002455,0.017990
17,rs61731104,1.448329,0.973007,0.035618,0.106314
...,...,...,...,...,...
39435,rs6002380,0.225938,0.152430,0.594377,0.703346
39454,rs12165508,0.082316,0.061067,0.827340,0.868025
39461,rs73439311,2.142339,1.440353,0.007205,0.036245
39467,rs2234058,0.925545,0.622003,0.118701,0.238559


In [110]:
df3[df3['skew_p'] <= 0.01]

,snp,c_skew_logp,c_skew_fdr,skew_p,stats_fdr
14,rs10910099,2.609976,1.744575,0.002455,0.017990
56,rs56178029,5.608785,3.551579,0.000002,0.000281
144,rs35883594,5.092412,3.264804,0.000008,0.000543
186,rs2457823,3.421907,2.257675,0.000379,0.005520
230,rs12044559,2.864907,1.906437,0.001365,0.012393
...,...,...,...,...,...
39126,rs7293064,2.106100,1.414579,0.007832,0.038461
39170,rs9610708,3.143543,2.078998,0.000719,0.008329
39316,rs6002851,2.130048,1.432578,0.007412,0.036900
39411,rs2272804,2.623654,1.754817,0.002379,0.017570


In [111]:
df3[df3['stats_fdr'] <= 0.05]

,snp,c_skew_logp,c_skew_fdr,skew_p,stats_fdr
14,rs10910099,2.609976,1.744575,0.002455,0.017990
56,rs56178029,5.608785,3.551579,0.000002,0.000281
144,rs35883594,5.092412,3.264804,0.000008,0.000543
186,rs2457823,3.421907,2.257675,0.000379,0.005520
230,rs12044559,2.864907,1.906437,0.001365,0.012393
...,...,...,...,...,...
39126,rs7293064,2.106100,1.414579,0.007832,0.038461
39170,rs9610708,3.143543,2.078998,0.000719,0.008329
39316,rs6002851,2.130048,1.432578,0.007412,0.036900
39411,rs2272804,2.623654,1.754817,0.002379,0.017570


In [112]:
#df3['difference'] = df3['c_skew_fdr'] - df3['stats_fdr']

In [113]:
df4 = df3[['snp','c_skew_logp', 'skew_p', 'c_skew_fdr', 'stats_fdr']]
df4.sort_values(by='c_skew_fdr')

,snp,c_skew_logp,skew_p,c_skew_fdr,stats_fdr
19946,rs114196109,0.001750,9.959775e-01,0.001350,0.995978
8035,rs1659087,0.001831,9.957923e-01,0.001350,0.995978
33987,rs143630869,0.002101,9.951741e-01,0.001499,0.995634
16411,rs1171614,0.002822,9.935228e-01,0.002045,0.994384
30010,rs146199012,0.002747,9.936948e-01,0.002045,0.994384
...,...,...,...,...,...
11605,rs2191501,7.402123,3.961654e-08,4.393105,0.000040
2271,rs9661285,7.422196,3.782722e-08,4.393105,0.000040
8432,rs2351011,7.331125,4.665256e-08,4.393105,0.000040
30970,rs11080327,8.557923,2.767432e-09,5.221964,0.000006


manual test

In [114]:
from math import log10

In [115]:
p = 0.077860
log_p = -log10(p)
print(log_p)

1.108685600617857


***

Benjamini hochberg

In [116]:
df3

,snp,c_skew_logp,c_skew_fdr,skew_p,stats_fdr
0,rs11548103,1.197350,0.803895,0.063482,0.156929
3,rs11102212,0.151356,0.102546,0.705738,0.788956
6,rs112338151,1.108686,0.743469,0.077860,0.180356
14,rs10910099,2.609976,1.744575,0.002455,0.017990
17,rs61731104,1.448329,0.973007,0.035618,0.106314
...,...,...,...,...,...
39435,rs6002380,0.225938,0.152430,0.594377,0.703346
39454,rs12165508,0.082316,0.061067,0.827340,0.868025
39461,rs73439311,2.142339,1.440353,0.007205,0.036245
39467,rs2234058,0.925545,0.622003,0.118701,0.238559


In [117]:
df3 = df3.sort_values(by='skew_p')

In [118]:
df3['rank'] = [*range(1, 4332)]

In [119]:
df3

,snp,c_skew_logp,c_skew_fdr,skew_p,stats_fdr,rank
3988,rs1617806,8.958733,5.321744,1.099681e-09,0.000005,1
30970,rs11080327,8.557923,5.221964,2.767432e-09,0.000006,2
2271,rs9661285,7.422196,4.393105,3.782722e-08,0.000040,3
11605,rs2191501,7.402123,4.393105,3.961654e-08,0.000040,4
8432,rs2351011,7.331125,4.393105,4.665256e-08,0.000040,5
...,...,...,...,...,...,...
16411,rs1171614,0.002822,0.002045,9.935228e-01,0.994384,4327
30010,rs146199012,0.002747,0.002045,9.936948e-01,0.994384,4328
33987,rs143630869,0.002101,0.001499,9.951741e-01,0.995634,4329
8035,rs1659087,0.001831,0.001350,9.957923e-01,0.995978,4330


In [120]:
bh = []
m = len(df3) #total amount of tests
for i in df3['rank']:
   critical_value =(i / m) * 0.05
   bh.append(critical_value)

In [121]:
df3['benhoch'] = bh

In [122]:
df3

,snp,c_skew_logp,c_skew_fdr,skew_p,stats_fdr,rank,benhoch
3988,rs1617806,8.958733,5.321744,1.099681e-09,0.000005,1,0.000012
30970,rs11080327,8.557923,5.221964,2.767432e-09,0.000006,2,0.000023
2271,rs9661285,7.422196,4.393105,3.782722e-08,0.000040,3,0.000035
11605,rs2191501,7.402123,4.393105,3.961654e-08,0.000040,4,0.000046
8432,rs2351011,7.331125,4.393105,4.665256e-08,0.000040,5,0.000058
...,...,...,...,...,...,...,...
16411,rs1171614,0.002822,0.002045,9.935228e-01,0.994384,4327,0.049954
30010,rs146199012,0.002747,0.002045,9.936948e-01,0.994384,4328,0.049965
33987,rs143630869,0.002101,0.001499,9.951741e-01,0.995634,4329,0.049977
8035,rs1659087,0.001831,0.001350,9.957923e-01,0.995978,4330,0.049988


***

#### Log2FC.

In [123]:
df_tewhey.loc[(df_tewhey['ref_log2fc'] >= 2)]


,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue,fdr,differentially_expressed
136,rs10910420,rs10910420,pos,ref,777.338742,4413.077004,2.103595,46.401528,41.805173,857.793656,...,0.272695,0.495684,0.356316,1.657845,1.116161,0.906207,1.566128e-42,2.175974e-86,0.076531,no
144,rs35883594,rs35883594,pos,ref,972.834942,4072.735029,2.019667,192.748518,188.152163,1078.011867,...,-1.264322,-1.393424,-1.312736,5.092412,3.264804,0.902434,7.044288e-189,1.224123e-19,0.000543,yes
300,rs41268482,rs41268482,pos,ref,1529.650957,19995.837650,3.313746,148.754303,144.157948,1251.205316,...,0.723347,0.525172,0.649031,5.640928,3.560241,1.222542,6.951069e-145,1.416873e-207,0.000275,yes
450,rs2019755_RC,rs2019755,neg,ref,818.824133,7707.979700,3.075949,236.298944,231.702588,778.475165,...,0.228581,0.249800,0.236538,3.987761,2.615590,1.051831,1.983405e-232,5.382523e-315,0.002423,yes
451,rs2019755_RC_alt,rs2019755,neg,alt,1040.031371,8244.571529,2.888763,322.264823,317.668468,1053.197537,...,0.197697,0.224120,0.207606,4.687217,3.032499,0.987499,2.145520e-318,5.382523e-315,0.000928,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36994,rs2295356_RC,rs2295356,neg,ref,1076.217148,13963.882430,3.257462,135.147570,130.551214,1042.830761,...,-0.087569,-0.102613,-0.093210,0.446766,0.297057,1.032015,2.810513e-131,4.075112e-107,0.504595,no
37361,rs72629024,rs72629024,pos,ref,600.132683,3284.750288,2.165506,85.124117,80.527761,899.294355,...,-0.262784,-0.359833,-0.299177,3.006479,1.986049,0.667337,2.966460e-81,2.429927e-96,0.010326,yes
37604,rs6062371,rs6062371,pos,ref,664.534760,3814.051504,2.432781,187.883144,183.286789,1120.231983,...,-0.392281,-0.413717,-0.400320,3.284831,2.162389,0.593212,5.166676e-184,1.431465e-219,0.006880,yes
37805,rs2425068_RC,rs2425068,neg,ref,1639.496079,10847.658750,2.519838,142.583052,137.986697,2056.788119,...,0.079242,-0.033199,0.037077,1.065225,0.716485,0.797115,1.031105e-138,7.065923e-92,0.192094,no


In [124]:
df_tewhey.loc[(lcl_analysis['alt_log2fc'] >=2)]

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue,fdr,differentially_expressed
136,rs10910420,rs10910420,pos,ref,777.338742,4413.077004,2.103595,46.401528,41.805173,857.793656,...,0.272695,0.495684,0.356316,1.657845,1.116161,0.906207,1.566128e-42,2.175974e-86,0.076531,no
300,rs41268482,rs41268482,pos,ref,1529.650957,19995.837650,3.313746,148.754303,144.157948,1251.205316,...,0.723347,0.525172,0.649031,5.640928,3.560241,1.222542,6.951069e-145,1.416873e-207,0.000275,yes
450,rs2019755_RC,rs2019755,neg,ref,818.824133,7707.979700,3.075949,236.298944,231.702588,778.475165,...,0.228581,0.249800,0.236538,3.987761,2.615590,1.051831,1.983405e-232,5.382523e-315,0.002423,yes
451,rs2019755_RC_alt,rs2019755,neg,alt,1040.031371,8244.571529,2.888763,322.264823,317.668468,1053.197537,...,0.197697,0.224120,0.207606,4.687217,3.032499,0.987499,2.145520e-318,5.382523e-315,0.000928,yes
535,rs28519456_RC,rs28519456,neg,ref,1399.124680,188267.556400,6.743227,322.264823,317.668468,1192.343989,...,-0.024803,-0.047786,-0.033421,2.285176,1.542503,1.173424,2.145520e-318,5.382523e-315,0.028675,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36994,rs2295356_RC,rs2295356,neg,ref,1076.217148,13963.882430,3.257462,135.147570,130.551214,1042.830761,...,-0.087569,-0.102613,-0.093210,0.446766,0.297057,1.032015,2.810513e-131,4.075112e-107,0.504595,no
37604,rs6062371,rs6062371,pos,ref,664.534760,3814.051504,2.432781,187.883144,183.286789,1120.231983,...,-0.392281,-0.413717,-0.400320,3.284831,2.162389,0.593212,5.166676e-184,1.431465e-219,0.006880,yes
37805,rs2425068_RC,rs2425068,neg,ref,1639.496079,10847.658750,2.519838,142.583052,137.986697,2056.788119,...,0.079242,-0.033199,0.037077,1.065225,0.716485,0.797115,1.031105e-138,7.065923e-92,0.192094,no
38514,rs11702929,rs11702929,pos,ref,985.720909,2212.509452,1.090592,69.802484,65.206129,1164.155900,...,1.320393,1.019353,1.207503,5.840931,3.670880,0.846726,6.221154e-66,9.775654e-248,0.000213,yes


In [125]:
lcl_analysis.loc[(lcl_analysis['ref_log2fc'] >= 2) | (lcl_analysis['alt_log2fc'] >=2)]


,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,alt_logp,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue
136,rs10910420,rs10910420,pos,ref,777.338742,4413.077004,2.103595,46.401528,41.805173,857.793656,...,90.258701,85.662346,0.272695,0.495684,0.356316,1.657845,1.116161,0.906207,1.566128e-42,2.175974e-86
144,rs35883594,rs35883594,pos,ref,972.834942,4072.735029,2.019667,192.748518,188.152163,1078.011867,...,23.508530,18.912175,-1.264322,-1.393424,-1.312736,5.092412,3.264804,0.902434,7.044288e-189,1.224123e-19
300,rs41268482,rs41268482,pos,ref,1529.650957,19995.837650,3.313746,148.754303,144.157948,1251.205316,...,211.445024,206.848669,0.723347,0.525172,0.649031,5.640928,3.560241,1.222542,6.951069e-145,1.416873e-207
450,rs2019755_RC,rs2019755,neg,ref,818.824133,7707.979700,3.075949,236.298944,231.702588,778.475165,...,318.865369,314.269014,0.228581,0.249800,0.236538,3.987761,2.615590,1.051831,1.983405e-232,5.382523e-315
451,rs2019755_RC_alt,rs2019755,neg,alt,1040.031371,8244.571529,2.888763,322.264823,317.668468,1053.197537,...,318.865369,314.269014,0.197697,0.224120,0.207606,4.687217,3.032499,0.987499,2.145520e-318,5.382523e-315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37361,rs72629024,rs72629024,pos,ref,600.132683,3284.750288,2.165506,85.124117,80.527761,899.294355,...,100.210762,95.614407,-0.262784,-0.359833,-0.299177,3.006479,1.986049,0.667337,2.966460e-81,2.429927e-96
37604,rs6062371,rs6062371,pos,ref,664.534760,3814.051504,2.432781,187.883144,183.286789,1120.231983,...,223.440574,218.844219,-0.392281,-0.413717,-0.400320,3.284831,2.162389,0.593212,5.166676e-184,1.431465e-219
37805,rs2425068_RC,rs2425068,neg,ref,1639.496079,10847.658750,2.519838,142.583052,137.986697,2056.788119,...,95.747186,91.150831,0.079242,-0.033199,0.037077,1.065225,0.716485,0.797115,1.031105e-138,7.065923e-92
38514,rs11702929,rs11702929,pos,ref,985.720909,2212.509452,1.090592,69.802484,65.206129,1164.155900,...,251.606209,247.009854,1.320393,1.019353,1.207503,5.840931,3.670880,0.846726,6.221154e-66,9.775654e-248


In [126]:
lcl_analysis[lcl_analysis['logskew_comb'] >=2]

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,alt_logp,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue
11605,rs2191501,rs2191501,pos,ref,1143.306328,2244.053935,0.951726,50.867329,46.270974,1281.564244,...,318.865369,314.269014,2.100388,2.580932,2.280592,7.402123,4.393105,0.892118,5.358285e-47,5.382523e-315
30970,rs11080327,rs11080327,pos,ref,1374.536239,2209.978176,0.659369,20.099172,15.502817,918.581377,...,318.865369,314.269014,3.199681,2.688394,3.007948,8.557923,5.221964,1.496369,3.141831e-16,5.382523e-315


#### Visualisation log2FC volcanoplot

In [127]:
def volcanoplot(df):
    copy = df.copy()
    color_ref, color_alt = get_color(copy)
    copy['color_ref'] = color_ref
    copy['color_alt'] = color_alt
    
    source = ColumnDataSource(copy)
    p1 = figure(title='Volcano plot log2FC ref allele', width=350, height=350,
                 x_axis_label='log2FC', y_axis_label='-log10 p-value')
    p1.circle(x= 'ref_log2fc', y='ref_logp', color='color_ref', source=source, )

    p2 = figure(title='Volcanoplot lof2FC alt allele', width=350, height=350,
                x_axis_label='log2FC', y_axis_label='-log10 p-value')
    p2.circle(x='alt_log2fc', y='alt_logp', color='color_alt', source= source)

    p3 = figure(title='Volcano plot log2FC ref allele - p adjusted', width=350, height=350,
                 x_axis_label='log2FC', y_axis_label='-log10 p-value')
    p3.circle(x= 'ref_log2fc', y='ref_logpadj', color='color_ref', source=source, )

    p4 = figure(title='Volcano plot log2FC alt allele - p adjusted', width=350, height=350,
                 x_axis_label='log2FC', y_axis_label='-log10 p-value')
    p4.circle(x= 'alt_log2fc', y='alt_logpadj', color='color_alt', source=source, )

    grid = gridplot([[p1, p2], [p3, p4]])
    #p = row(p1, p2)
    show(grid)

In [128]:
def get_color(df):
    color_ref = ['blue' if x < 0 else 'red' for x in df['ref_log2fc']]
    color_alt = ['blue' if x < 0 else 'red' for x in df['alt_log2fc']]
    return color_ref, color_alt

In [129]:
def volcano_plot(df):
    df_copy = df.copy()
    color_ref, color_alt = get_color(df_copy)
    df_copy['color_ref'] = color_ref
    df_copy['color_alt'] = color_alt
    df_copy = df_copy[['ref_log2fc', 'alt_log2fc', 'ref_logp','ref_logpadj', 'alt_logp', 'alt_logpadj', 'color_ref', 'color_alt']]
    source = ColumnDataSource(df_copy)
    plots= []
    for i in df_copy.columns[2:-2]:
        p = figure(width=350, height=350)
        if i.startswith('ref'):
            p.circle(x='ref_log2fc', y=i, source=source, color='color_ref')
            plots.append(p)
        else:
           p.circle(x='alt_log2fc', y=i, source=source, color='color_alt')
           plots.append(p)
        #plots.append(p)
        #show(p)
    show(grid(plots, nrows=2))


In [130]:
df_tewhey.columns

Index(['id', 'snp', 'direction', 'haplotype', 'ref_ctrl_mean', 'ref_exp_mean',
       'ref_log2fc', 'ref_logp', 'ref_logpadj', 'alt_ctrl_mean',
       'alt_exp_mean', 'alt_log2fc', 'alt_logp', 'alt_logpadj',
       'logskew_12878', 'logskew_19239', 'logskew_comb', 'c_skew_logp',
       'c_skew_fdr', 'beta', 'ref_pvalue', 'alt_pvalue', 'fdr',
       'differentially_expressed'],
      dtype='object')

In [131]:
volcano_plot(df_tewhey)

In [132]:
volcanoplot(df_tewhey)

***
df.plot()

In [133]:
sort_reflogp = df_tewhey.sort_values(by='ref_logp')
#last value of the seventh column
sort_reflogp.iloc[-1,7]

322.2648227

In [134]:
sort_altlogp = df_tewhey.sort_values(by='alt_logp')
sort_altlogp.iloc[-1,12]

318.8653692

In [135]:
max_a = df_tewhey[df_tewhey['ref_logp'] == sort_reflogp.iloc[-1,7]]
len(max_a)

34

In [136]:
max_b = df_tewhey[df_tewhey['alt_logp'] == sort_altlogp.iloc[-1,12]]
len(max_b)

40

take out max values and 0's

In [137]:
out_max =df_tewhey[(df_tewhey['alt_logp'] != sort_altlogp.iloc[-1,12]) & (df_tewhey['ref_logp'] != sort_reflogp.iloc[-1,7]) & (df_tewhey['ref_logpadj'] != 0) & (df_tewhey['alt_logpadj'] != 0)]

In [138]:
volcano_plot(out_max)

In [139]:
volcanoplot(out_max)

In [140]:
df_tewhey['skewp'] = df_tewhey['c_skew_logp'].map(lambda x: 10 ** -x)

In [141]:
de = df_tewhey[df_tewhey['skewp'] <= 0.01]

In [142]:
volcano_plot(de)

### Histograms

In [143]:
hist_ref = df_tewhey.hvplot.hist('ref_ctrl_mean')
hist_ref

:Histogram   [ref_ctrl_mean]   (ref_ctrl_mean_count)

In [144]:
hist_alt = df_tewhey.hvplot.hist('alt_ctrl_mean')
hist_alt

:Histogram   [alt_ctrl_mean]   (alt_ctrl_mean_count)

In [145]:
hvplot.help('hist', generic=False, style=False)


A `histogram` displays an approximate representation of the distribution of continuous data.

Reference: https://hvplot.holoviz.org/reference/pandas/hist.html

Parameters
----------
y : string or sequence
    Field(s) in the *wide* data to compute the distribution(s) from.
    Please note the fields should contain continuous data. Not categorical.
by : string or sequence
    Field(s) in the *long* data to group by.
bins : int, optional
    The number of bins
bin_range: tuple, optional
    The lower and upper range of the bins. Default is None.
normed : bool, optional
    If True the distribution will sum to 1. Default is False.
cumulative: bool, optional
    If True, then a histogram is computed where each bin gives the counts in that bin plus
    all bins for smaller values. The last bin gives the total number of datapoints.
    Default is False.
alpha : float, optional
    An alpha value between 0.0 and 1.0 to better visualize multiple fields. Default is 1.0.
kwds : optional
    Add

In [147]:
df_tewhey.hvplot.hist('ref_ctrl_mean', alpha=0.6) * df_tewhey.hvplot.hist('alt_ctrl_mean', alpha=0.6)

:Overlay
   .Histogram.I  :Histogram   [ref_ctrl_mean]   (ref_ctrl_mean_count)
   .Histogram.II :Histogram   [alt_ctrl_mean]   (alt_ctrl_mean_count)

In [148]:
df_tewhey.hvplot.kde('ref_ctrl_mean') * df_tewhey.hvplot.kde('alt_ctrl_mean')

:Overlay
   .Distribution.I  :Distribution   [ref_ctrl_mean]   (Density)
   .Distribution.II :Distribution   [alt_ctrl_mean]   (Density)

In [ ]:
df_tewhey.to_csv('lcl_analysis1.csv', sep=';')

In [ ]:
df_tewhey.to_excel('lcl_analysis_output.xlsx')

In [ ]:
df_tewhey[df_tewhey['alt_pvalue'] == 1]

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue,fdr,DE,skewp
14,rs10910099_RC,rs10910099,neg,ref,1565.369298,2410.813917,0.609195,18.726474,14.130119,1597.275307,...,-0.442277,-0.365843,2.609976,1.744575,0.980025,7.411069e-15,1.0,0.018006,yes,0.002455
17,rs61731104,rs61731104,pos,ref,787.310108,1106.543203,0.472118,7.158719,2.562364,851.116339,...,-0.181428,-0.334618,1.448329,0.973007,0.925032,2.739278e-03,1.0,0.106413,no,0.035618
37,rs6688187_RC,rs6688187,neg,ref,962.823406,1313.143420,0.434006,6.760603,2.164248,855.198391,...,-0.320990,-0.202553,1.094694,0.735315,1.125848,6.850972e-03,1.0,0.183944,no,0.080409
108,rs57577924_RC,rs57577924,neg,ref,750.915419,1184.321454,0.628851,10.365556,5.769201,487.199529,...,-0.212974,-0.283369,1.077220,0.723985,1.541289,1.701373e-06,1.0,0.188806,no,0.083710
253,rs12140585_RC,rs12140585,neg,ref,880.199885,1215.487885,0.442313,7.583456,2.987101,596.405690,...,-0.188095,-0.175171,0.642931,0.429914,1.475841,1.030146e-03,1.0,0.371609,no,0.227546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39222,rs5753095_RC,rs5753095,neg,ref,758.642972,1175.613684,0.531981,15.985203,11.388848,589.611562,...,-0.514704,-0.176991,0.746790,0.501441,1.286683,4.084626e-12,1.0,0.315180,no,0.179147
39270,rs12168499_RC,rs12168499,neg,ref,781.682780,1252.717602,0.656405,13.151322,8.554967,692.382725,...,-0.198476,-0.333445,0.999313,0.670461,1.128975,2.786336e-09,1.0,0.213569,no,0.100158
39411,rs2272804,rs2272804,pos,ref,1249.354490,1633.960228,0.371340,9.337950,4.741594,1245.604249,...,-0.561062,-0.446172,2.623654,1.754817,1.003011,1.813033e-05,1.0,0.017587,yes,0.002379
39412,rs2272804_RC,rs2272804,neg,ref,1205.182141,1535.483100,0.335348,10.714907,6.118552,1619.957695,...,-0.280289,-0.160401,0.913794,0.614838,0.743959,7.611107e-07,1.0,0.242751,no,0.121957


***

### Get chromosome positions for rsid's

In [ ]:
genome_position_nexus = pd.read_csv('gen_coords_4563dd52.txt', sep='\t')

In [ ]:
biomart_rsid = pd.read_csv('mart_export.txt')

In [ ]:
biomart_rsid

,Variant name,Variant source,Chromosome/scaffold name,Chromosome/scaffold position start (bp),Chromosome/scaffold position end (bp),Synonym name
0,rs11810220,dbSNP,1,163311300,163311300,NaN
1,rs11585048,dbSNP,1,2602648,2602648,rs59642996
2,rs11585048,dbSNP,HSCHR1_1_CTG3,2602648,2602648,rs59642996
3,rs11585844,dbSNP,1,37563668,37563668,NaN
4,rs11587500,dbSNP,1,24190390,24190390,rs17184644
...,...,...,...,...,...,...
7902,rs111980103,dbSNP,16,970874,970874,RCV001667153
7903,rs111980103,dbSNP,16,970874,970874,RCV002421238
7904,rs59522292,dbSNP,12,124914185,124914185,NaN
7905,rs56812038,dbSNP,7,32681035,32681035,NaN


In [ ]:
biomart_rsid.columns = [i.lower().replace(' ', '_') for i in biomart_rsid.columns]

In [ ]:
biomart_rsid['variant_name'].value_counts()

rs9332739     266
rs1063355     133
rs1071630     126
rs878886       99
rs1130422      98
             ... 
rs12791871      1
rs2328393       1
rs10823267      1
rs12790049      1
rs59687507      1
Name: variant_name, Length: 2835, dtype: int64

In [ ]:
rs9332739 = biomart_rsid[biomart_rsid['variant_name']== 'rs9332739']
rs9332739

,variant_name,variant_source,chromosome/scaffold_name,chromosome/scaffold_position_start_(bp),chromosome/scaffold_position_end_(bp),synonym_name
4007,rs9332739,dbSNP,6,31936027,31936027,rs52807427
4008,rs9332739,dbSNP,6,31936027,31936027,rs111255853
4009,rs9332739,dbSNP,6,31936027,31936027,rs114895340
4010,rs9332739,dbSNP,6,31936027,31936027,rs140174264
4011,rs9332739,dbSNP,6,31936027,31936027,NM_000063.4:c.954G>A
...,...,...,...,...,...,...
4268,rs9332739,dbSNP,HSCHR6_MHC_MANN_CTG1,31975397,31975397,VAR_019158
4269,rs9332739,dbSNP,HSCHR6_MHC_MANN_CTG1,31975397,31975397,C2base_D0094:g.9059G>C
4270,rs9332739,dbSNP,HSCHR6_MHC_MANN_CTG1,31975397,31975397,RCV001516299
4271,rs9332739,dbSNP,HSCHR6_MHC_MANN_CTG1,31975397,31975397,613927.0004


In [ ]:
rs9332739['chromosome/scaffold_position_start_(bp)'].value_counts()

31936027    38
31923573    38
31926408    38
31918213    38
31928681    38
32012470    38
31975397    38
Name: chromosome/scaffold_position_start_(bp), dtype: int64

In [ ]:
rsid_biomart = list(biomart_rsid['variant_name'].unique())

In [ ]:
difference_rsid_biomart = list(set(unique_rs_id) - set(rsid_biomart))
len(difference_rsid_biomart)

542

***
merging dataframes

In [ ]:
genome_position_nexus

,Variation ID,dbSNP,Chromosome,Position,REF Allele,ALT Allele (IUPAC),Minor Allele,Minor Allele Global Frequency,Contig,Contig Position,Band
0,rs148649543,rs148649543,1,752796,C,T,None,None,GL000003.1,231428,p36.33
1,rs2275915,rs2275915,1,1342612,G,Y,C,0.430312,GL000003.1,821244,p36.33
2,rs72634819,rs72634819,1,1596500,C,T,T,0.116214,GL000003.1,1075132,p36.33
3,rs6699975,rs6699975,1,1603434,G,W,T,0.256789,GL000003.1,1082066,p36.33
4,rs7544851,rs7544851,1,1610524,T,C,C,0.340855,GL000003.1,1089156,p36.33
...,...,...,...,...,...,...,...,...,...,...,...
2831,rs9330192,rs9330192,9,140161334,C,K,C,0.379792,GL000092.1,944337,q34.3
2832,rs77186230,rs77186230,9,140179540,C,W,A,0.161941,GL000092.1,962543,q34.3
2833,rs4962240,rs4962240,9,140356374,C,D,A,0.415735,GL000092.1,1139377,q34.3
2834,rs7028824,rs7028824,9,140656536,A,G,G,0.078474,GL000092.1,1439539,q34.3


In [ ]:
genome_position_nexus.columns = [i.lower() for i in genome_position_nexus.columns]

In [ ]:
position = genome_position_nexus[['dbsnp', 'chromosome', 'position']]
position = position.rename(columns={'dbsnp': 'snp'})

In [ ]:
snps_nexus = list(position['snp'])

In [ ]:
df_tewhey['snp'].isin(snps_nexus).sum()

3207

In [ ]:
df_result = pd.merge(df_tewhey, position,how='outer',on=['snp'])

In [ ]:
df_result.snp.value_counts()

rs117592340         4
rs117688250         4
chr17:43775212:D    4
rs112646635         4
rs117452327         4
                   ..
rs1047207           1
rs56089143          1
rs10081322          1
chr7:127142915:I    1
rs2510053           1
Name: snp, Length: 3595, dtype: int64

In [ ]:
df_result

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue,chromosome,position
0,rs11548103_RC,rs11548103,neg,ref,893.147913,1403.234147,0.637129,20.726159,16.129804,985.132571,...,-0.157649,-0.070399,-0.124930,1.197350,0.803895,0.906627,7.416455e-17,2.275395e-20,1.0,153588340.0
1,rs11102212_RC,rs11102212,neg,ref,272.682393,724.187989,1.276380,20.903079,16.306723,270.606119,...,-0.182603,0.057417,-0.092596,0.151356,0.102546,1.007673,4.934880e-17,2.090037e-14,1.0,111642031.0
2,rs112338151,rs112338151,pos,ref,311.022339,938.600426,1.454065,35.570382,30.974027,441.094084,...,-0.149852,-0.070538,-0.120109,1.108686,0.743469,0.705116,1.061629e-31,5.410694e-43,1.0,26698747.0
3,rs10910099_RC,rs10910099,neg,ref,1565.369298,2410.813917,0.609195,18.726474,14.130119,1597.275307,...,-0.319983,-0.442277,-0.365843,2.609976,1.744575,0.980025,7.411069e-15,1.000000e+00,1.0,2533552.0
4,rs61731104,rs61731104,pos,ref,787.310108,1106.543203,0.472118,7.158719,2.562364,851.116339,...,-0.426533,-0.181428,-0.334618,1.448329,0.973007,0.925032,2.739278e-03,1.000000e+00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4334,rs6002380_RC,rs6002380,neg,ref,1021.939757,1393.401924,0.436551,7.957672,3.361317,1410.403292,...,-0.072219,-0.194436,-0.118050,0.225938,0.152430,0.724573,4.351945e-04,1.000000e+00,22.0,41821423.0
4335,rs12165508_RC,rs12165508,neg,ref,905.677775,1186.698469,0.370023,10.844473,6.248118,602.818906,...,-0.004909,0.002897,-0.001982,0.082316,0.061067,1.502404,5.647837e-07,9.411186e-02,22.0,41819960.0
4336,rs73439311_RC,rs73439311,neg,ref,225.047471,518.727303,1.021873,15.889120,11.292765,264.268231,...,-0.768413,-0.555736,-0.688659,2.142339,1.440353,0.851587,5.096070e-12,4.791545e-02,22.0,50753334.0
4337,rs2234058,rs2234058,pos,ref,673.043758,1373.481543,0.938229,34.201737,29.605382,587.687732,...,0.089103,0.123557,0.102023,0.925545,0.622003,1.145240,2.480950e-30,1.707459e-33,22.0,41777883.0


***
Check missing positions

In [ ]:
nan = df_result[df_result['position'].isna()]

In [ ]:
rs_nan = [i for i in nan['snp'] if i.startswith('rs')]

In [ ]:
list(set(rs_nan) & set(snps_nexus))

[]

In [ ]:
unique_nan_rs = set(rs_nan)

Overlap between rsid's missing previous query on snp nexus and ensmbl biomart results

In [ ]:
list(set(rsid_biomart) & unique_nan_rs)

['rs2125313', 'rs61731104', 'rs6002527', 'rs2192680', 'rs7318257']

overlap snp nexus results and ensmbl biomart

In [ ]:
biomart_nexus = list(set(rsid_biomart) & set(snps_nexus))
len(biomart_nexus)

2830

In [ ]:
missing_rsid_biomart = list(set(unique_rs_id) - set(rsid_biomart))
len(missing_rsid_biomart)

542

In [ ]:
missing_rsid_nexus = list(set(unique_rs_id) - set(snps_nexus))
len(missing_rsid_nexus)

543

In [ ]:
with open ('missing_biomart.txt', 'w') as file:
    for id in missing_rsid_biomart:
        file.write(id + '\n')
    file.write('rs62062325')
# 'dbsnp\t' + 

In [ ]:
#overlap missing rsid's of query'schmidt_crispr
overlap_biomart_nexus = list(set(missing_rsid_biomart) & set(missing_rsid_nexus))
difference_biomart_nexus = list(set(missing_rsid_biomart) - set(missing_rsid_nexus))
difference_nexus_biomart = list(set(missing_rsid_nexus) - set(missing_rsid_biomart))
f'Overlap {len(overlap_biomart_nexus)}, difference biomart-nexus: {len(difference_biomart_nexus)}, difference nexus-biomart: {len(difference_nexus_biomart)}'

'Overlap 538, difference biomart-nexus: 4, difference nexus-biomart: 5'

In [ ]:
difference_nexus_biomart
# rs6002527 22:41901964 (GRCh38) | 22:42297968 (GRCh37)
# rs2192680 7:38354543 (GRCh38) | 7:38394144 (GRCh37)
# rs61731104 1:181050066 (GRCh38) | 1:181019202 (GRCh37)
# rs7318257 13:21344884 (GRCh38) | 13:21919023 (GRCh37)
# rs2125313 4:39456431 (GRCh38) | 4:39458051 (GRCh37)

['rs2125313', 'rs61731104', 'rs6002527', 'rs2192680', 'rs7318257']

In [ ]:
for i in difference_nexus_biomart:
    df_selection = position[position['snp'] == i]
df_selection

,snp,chromosome,position


In [ ]:
difference_biomart_nexus

['rs10906961', 'rs139276563', 'rs12952492', 'rs192096839']

In [ ]:
#biomart_rsid[biomart_rsid['variant_name'] == 'rs116971499']
df_tewhey[df_tewhey['snp'] == 'rs116971499']
#position[position['snp'] == 'rs116971499']

#dbsnp merged into rs62062325 (snp nexus does find this one)

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,alt_logp,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue
26781,rs116971499_RC,rs116971499,neg,ref,652.203064,2392.060939,1.828366,134.089304,129.492949,1203.004406,...,90.445630,85.849275,-0.177367,-0.338685,-0.237861,1.798026,1.204399,0.542145,3.214039e-130,1.414897e-86
26782,rs116971499_alt,rs116971499,pos,alt,814.917435,1192.806042,0.537783,14.947365,10.351010,879.474050,...,3.671251,0.000000,-0.218930,-0.238637,-0.226320,1.026404,0.689140,0.926596,4.456462e-11,1.000000e+00
26783,rs116971499_RC_alt,rs116971499,neg,alt,1655.366696,9032.323903,2.370641,186.487974,181.891619,1066.640875,...,207.233642,202.637287,0.259127,0.276683,0.265710,3.354889,2.220333,1.551944,1.283457e-182,2.305224e-203


In [ ]:
position[(position['snp'] == 'rs192096839') | (position['snp'] == 'rs12952492')| (position['snp'] == 'rs10906961')| (position['snp'] == 'rs139276563')]


,snp,chromosome,position
340,rs10906961,10,47994460
853,rs139276563,14,106549338
1144,rs12952492,17,34932498
1849,rs192096839,22,24366720


dbsnp: <br>
- rs115444759 no results
- rs139276563 (no mapping)
- rs10906961 (no mapping)
- rs12952492 NT_187614.1:811563 (GRCh38) | NT_187614.1:34932498 (GRCh37)
- rs192096839 NT_187633.1:260895 (GRCh38) | NT_187633.1:24366720 (GRCh37)
    - rs754370807 has merged into rs192096839 (same position) | NT_187633.1:260895 (GRCh38) | NT_187633.1:24366720 (GRCh37)



In [ ]:
position

,snp,chromosome,position
0,rs148649543,1,752796
1,rs2275915,1,1342612
2,rs72634819,1,1596500
3,rs6699975,1,1603434
4,rs7544851,1,1610524
...,...,...,...
2831,rs9330192,9,140161334
2832,rs77186230,9,140179540
2833,rs4962240,9,140356374
2834,rs7028824,9,140656536


***

In [ ]:
synonym_biomart = pd.read_csv('mart_export_synonym.txt')

In [ ]:
synonym_biomart.columns = [i.lower().replace(' ', '_') for i in synonym_biomart.columns]

In [ ]:
synonym_biomart

,variant_name,variant_source,chromosome/scaffold_name,chromosome/scaffold_position_start_(bp),chromosome/scaffold_position_end_(bp),synonym_name
0,rs2356416,dbSNP,1,45567593,45567593,rs74785550
1,rs7534581,dbSNP,1,1659114,1659114,rs9661285
2,rs112868731,dbSNP,3,41841863,41841863,rs144523572
3,rs111248130,dbSNP,4,55242231,55242231,rs145051830
4,rs55993837,dbSNP,3,41850539,41850539,rs140970237
...,...,...,...,...,...,...
1927,rs2885047,dbSNP,9,470259,470259,rs113679677
1928,rs4301823,dbSNP,12,57792811,57792811,rs56261123
1929,rs13221668,dbSNP,7,73763368,73763368,rs74539570
1930,rs4000157,dbSNP,7,32730090,32730090,rs145127609


In [ ]:
len(missing_rsid_biomart)

542

In [ ]:
synonym_biomart['variant_name'].value_counts()

rs77550992     8
rs3928943      8
rs2916809      8
rs7747114      8
rs9260555      8
              ..
rs2439         1
rs55993837     1
rs111248130    1
rs112868731    1
rs77718176     1
Name: variant_name, Length: 541, dtype: int64

In [ ]:
def change_column_names(dataframe):
    dataframe.columns = [i.lower().replace(' ', '_') for i in dataframe.columns]

In [ ]:
total_synonym = pd.read_csv('mart_export_total_synonym.txt')
total_variant_name_biomart = pd.read_csv('mart_export_total_variantname.txt')

In [ ]:
total_synonym

,Variant name,Variant source,Chromosome/scaffold name,Chromosome/scaffold position start (bp),Chromosome/scaffold position end (bp),Synonym name
0,rs2356416,dbSNP,1,45567593,45567593,rs74785550
1,rs7534581,dbSNP,1,1659114,1659114,rs9661285
2,rs112868731,dbSNP,3,41841863,41841863,rs144523572
3,rs111248130,dbSNP,4,55242231,55242231,rs145051830
4,rs55993837,dbSNP,3,41850539,41850539,rs140970237
...,...,...,...,...,...,...
1927,rs2885047,dbSNP,9,470259,470259,rs113679677
1928,rs4301823,dbSNP,12,57792811,57792811,rs56261123
1929,rs13221668,dbSNP,7,73763368,73763368,rs74539570
1930,rs4000157,dbSNP,7,32730090,32730090,rs145127609


In [ ]:
change_column_names(total_synonym)
change_column_names(total_variant_name_biomart)

In [ ]:
#total_variant_name_biomart['variant_name'].value_counts()

In [ ]:
total_synonym['synonym_name'].value_counts()

rs115317399    8
rs116537579    8
rs115799879    8
rs114679871    8
rs114788734    8
              ..
rs76542740     1
rs140970237    1
rs145051830    1
rs144523572    1
rs113859809    1
Name: synonym_name, Length: 539, dtype: int64

In [ ]:
len(total_variant_name_biomart['variant_name'].value_counts()) + len(total_synonym['synonym_name'].value_counts())

3374

In [ ]:
total_synonym['synonym_name'].isna().sum()

0

In [ ]:
len(total_synonym['synonym_name'].unique()) + len(total_variant_name_biomart['variant_name'].unique())

3374

In [ ]:
f'overlap filter variant name and variant synonym {list(set(total_variant_name_biomart.variant_name.unique()) & set(total_synonym.variant_name.unique()))}'

'overlap filter variant name and variant synonym []'

In [ ]:
df_biomart = pd.concat([total_variant_name_biomart, total_synonym])

In [ ]:
df_biomart

,variant_name,variant_source,chromosome/scaffold_name,chromosome/scaffold_position_start_(bp),chromosome/scaffold_position_end_(bp),synonym_name
0,rs11810220,dbSNP,1,163311300,163311300,NaN
1,rs11585048,dbSNP,1,2602648,2602648,rs59642996
2,rs11585048,dbSNP,HSCHR1_1_CTG3,2602648,2602648,rs59642996
3,rs11585844,dbSNP,1,37563668,37563668,NaN
4,rs11587500,dbSNP,1,24190390,24190390,rs17184644
...,...,...,...,...,...,...
1927,rs2885047,dbSNP,9,470259,470259,rs113679677
1928,rs4301823,dbSNP,12,57792811,57792811,rs56261123
1929,rs13221668,dbSNP,7,73763368,73763368,rs74539570
1930,rs4000157,dbSNP,7,32730090,32730090,rs145127609


In [ ]:
#df_biomart

In [ ]:
synonym_biomart['synonym_name'].value_counts()

rs115317399    8
rs116537579    8
rs115799879    8
rs114679871    8
rs114788734    8
              ..
rs76542740     1
rs140970237    1
rs145051830    1
rs144523572    1
rs113859809    1
Name: synonym_name, Length: 539, dtype: int64

In [ ]:
synonym_biomart[synonym_biomart['synonym_name'] == 'rs115317399']

,variant_name,variant_source,chromosome/scaffold_name,chromosome/scaffold_position_start_(bp),chromosome/scaffold_position_end_(bp),synonym_name
1390,rs2975042,dbSNP,6,29952759,29952759,rs115317399
1391,rs2975042,dbSNP,HSCHR6_MHC_COX_CTG1,29942305,29942305,rs115317399
1392,rs2975042,dbSNP,HSCHR6_MHC_QBL_CTG1,29942201,29942201,rs115317399
1393,rs2975042,dbSNP,HSCHR6_MHC_DBB_CTG1,29942740,29942740,rs115317399
1394,rs2975042,dbSNP,HSCHR6_MHC_SSTO_CTG1,29942032,29942032,rs115317399
1395,rs2975042,dbSNP,HSCHR6_MHC_MANN_CTG1,29947927,29947927,rs115317399
1396,rs2975042,dbSNP,HSCHR6_MHC_MCF_CTG1,30031267,30031267,rs115317399
1397,rs2975042,dbSNP,HSCHR6_MHC_APD_CTG1,29944918,29944918,rs115317399


In [ ]:
synonym_biomart[synonym_biomart['variant_name'] == 'rs77550992']

,variant_name,variant_source,chromosome/scaffold_name,chromosome/scaffold_position_start_(bp),chromosome/scaffold_position_end_(bp),synonym_name
1913,rs77550992,dbSNP,6,29856738,29856738,rs116671664
1914,rs77550992,dbSNP,HSCHR6_MHC_COX_CTG1,29853924,29853924,rs116671664
1915,rs77550992,dbSNP,HSCHR6_MHC_QBL_CTG1,29853684,29853684,rs116671664
1916,rs77550992,dbSNP,HSCHR6_MHC_DBB_CTG1,29853937,29853937,rs116671664
1917,rs77550992,dbSNP,HSCHR6_MHC_SSTO_CTG1,29853726,29853726,rs116671664
1918,rs77550992,dbSNP,HSCHR6_MHC_MANN_CTG1,29853267,29853267,rs116671664
1919,rs77550992,dbSNP,HSCHR6_MHC_MCF_CTG1,29853585,29853585,rs116671664
1920,rs77550992,dbSNP,HSCHR6_MHC_APD_CTG1,29856544,29856544,rs116671664


In [ ]:
import re

In [ ]:
# for i in total_synonym['chromosome/scaffold_name']:
#     if re.match('^(\d{1,2})', i):
#         print(i)

In [ ]:
total_synonym[total_synonym['synonym_name'] == 'rs116671664']

,variant_name,variant_source,chromosome/scaffold_name,chromosome/scaffold_position_start_(bp),chromosome/scaffold_position_end_(bp),synonym_name
1913,rs77550992,dbSNP,6,29856738,29856738,rs116671664
1914,rs77550992,dbSNP,HSCHR6_MHC_COX_CTG1,29853924,29853924,rs116671664
1915,rs77550992,dbSNP,HSCHR6_MHC_QBL_CTG1,29853684,29853684,rs116671664
1916,rs77550992,dbSNP,HSCHR6_MHC_DBB_CTG1,29853937,29853937,rs116671664
1917,rs77550992,dbSNP,HSCHR6_MHC_SSTO_CTG1,29853726,29853726,rs116671664
1918,rs77550992,dbSNP,HSCHR6_MHC_MANN_CTG1,29853267,29853267,rs116671664
1919,rs77550992,dbSNP,HSCHR6_MHC_MCF_CTG1,29853585,29853585,rs116671664
1920,rs77550992,dbSNP,HSCHR6_MHC_APD_CTG1,29856544,29856544,rs116671664


In [ ]:
chr_name = [i for i in total_synonym['chromosome/scaffold_name'].unique() if re.match('^(\d{1,2})', i)]

In [ ]:
unique_syn = total_synonym[total_synonym['chromosome/scaffold_name'].isin(chr_name)]


In [ ]:
unique_syn

,variant_name,variant_source,chromosome/scaffold_name,chromosome/scaffold_position_start_(bp),chromosome/scaffold_position_end_(bp),synonym_name
0,rs2356416,dbSNP,1,45567593,45567593,rs74785550
1,rs7534581,dbSNP,1,1659114,1659114,rs9661285
2,rs112868731,dbSNP,3,41841863,41841863,rs144523572
3,rs111248130,dbSNP,4,55242231,55242231,rs145051830
4,rs55993837,dbSNP,3,41850539,41850539,rs140970237
...,...,...,...,...,...,...
1927,rs2885047,dbSNP,9,470259,470259,rs113679677
1928,rs4301823,dbSNP,12,57792811,57792811,rs56261123
1929,rs13221668,dbSNP,7,73763368,73763368,rs74539570
1930,rs4000157,dbSNP,7,32730090,32730090,rs145127609


In [ ]:
unique_syn[unique_syn['synonym_name'] == 'rs116671664']

,variant_name,variant_source,chromosome/scaffold_name,chromosome/scaffold_position_start_(bp),chromosome/scaffold_position_end_(bp),synonym_name
1913,rs77550992,dbSNP,6,29856738,29856738,rs116671664


In [ ]:
total_chr_name = [i for i in total_variant_name_biomart['chromosome/scaffold_name'] if re.match('^(\d{1,2})', i)]
unique_chr_total = total_variant_name_biomart[total_variant_name_biomart['chromosome/scaffold_name'].isin(total_chr_name)]
unique_chr_total

,variant_name,variant_source,chromosome/scaffold_name,chromosome/scaffold_position_start_(bp),chromosome/scaffold_position_end_(bp),synonym_name
0,rs11810220,dbSNP,1,163311300,163311300,NaN
1,rs11585048,dbSNP,1,2602648,2602648,rs59642996
3,rs11585844,dbSNP,1,37563668,37563668,NaN
4,rs11587500,dbSNP,1,24190390,24190390,rs17184644
5,rs11587500,dbSNP,1,24190390,24190390,rs59459702
...,...,...,...,...,...,...
7902,rs111980103,dbSNP,16,970874,970874,RCV001667153
7903,rs111980103,dbSNP,16,970874,970874,RCV002421238
7904,rs59522292,dbSNP,12,124914185,124914185,NaN
7905,rs56812038,dbSNP,7,32681035,32681035,NaN


In [ ]:
#unique_chr_total = unique_chr_total.rename(columns={'variant_name': 'snp','chromosome/scaffold_name': 'snp_chromosome', 'chromosome/scaffold_position_start_(bp)': 'snp_position'})

In [ ]:
def format_dataframe(df):
    df = df.rename(columns={'variant_name': 'snp','chromosome/scaffold_name': 'snp_chromosome', 'chromosome/scaffold_position_start_(bp)': 'snp_position'})
    df = df.drop(columns=['variant_source', 'chromosome/scaffold_position_end_(bp)'])
    return df

In [ ]:
unique_chr_total = format_dataframe(unique_chr_total)

In [ ]:
unique_chr_total['snp'].value_counts()

rs3742801     78
rs1800734     59
rs4802741     49
rs2298699     44
rs6915401     43
              ..
rs7123480      1
rs2910688      1
rs7125402      1
rs2915876      1
rs59687507     1
Name: snp, Length: 2835, dtype: int64

In [ ]:
# new_chr_names = []
# for i in unique_chr_total['chr']:
#     chr_nr = 'chr' + i
#     new_chr_names.append(chr_nr)
# unique_chr_total['chr'] = new_chr_names
# unique_chr_total

In [ ]:
# unique_syn

In [ ]:
#unique_chr_total['variant_name'].unique()


In [ ]:
#unique_chr_total[unique_chr_total['variant_name']]

In [ ]:
#unique_chr_total[unique_chr_total['variant_name']=='rs3742801']

In [ ]:
# unique_chr_total['variant_name'].value_counts()

In [ ]:
#df_chr

In [ ]:
#merged

In [ ]:
#genome_position[genome_position['dbsnp'] == 'rs2192680']

In [ ]:
with open ('rsid_nan.txt', 'w') as file:
    for id in unique_nan_rs:
        file.write('dbsnp\t' +id + '\n')

In [ ]:
df_rs = df_tewhey[df_tewhey['snp'].isin(snp)]

In [ ]:
df_rs

,id,snp,direction,haplotype,ref_ctrl_mean,ref_exp_mean,ref_log2fc,ref_logp,ref_logpadj,alt_ctrl_mean,...,alt_logp,alt_logpadj,logskew_12878,logskew_19239,logskew_comb,c_skew_logp,c_skew_fdr,beta,ref_pvalue,alt_pvalue
34,chr1:150824527:I_RC,chr1:150824527:I,neg,ref,665.673312,928.793886,0.467121,7.948176,3.351821,616.995944,...,5.977152,1.380797,-0.149947,0.169805,-0.030040,0.385947,0.256515,1.078894,4.448146e-04,4.161048e-02
331,chr1:205753876:D_RC,chr1:205753876:D,neg,ref,1420.436964,3614.404655,1.317902,87.999640,83.403285,888.001576,...,37.710249,33.113894,-0.365522,-0.288343,-0.336580,2.764688,1.837817,1.599588,3.951077e-84,7.693188e-34
402,chr1:171993854:D,chr1:171993854:D,pos,ref,1274.639199,1766.597922,0.437787,11.528658,6.932303,1447.944789,...,10.483445,5.887090,0.237896,-0.094620,0.113202,0.510838,0.340568,0.880309,1.168683e-07,1.296911e-06
700,chr1:41499967:I,chr1:41499967:I,pos,ref,978.055615,1954.770916,0.954079,62.441074,57.844719,1054.557178,...,18.463807,13.867452,-0.207228,-0.232217,-0.216599,0.715108,0.480724,0.927456,1.429820e-58,1.356900e-14
717,chr1:22352395:D,chr1:22352395:D,pos,ref,1328.979443,9369.413644,2.676462,283.389681,278.793326,3635.588956,...,277.421672,272.825317,-0.016826,-0.086135,-0.042817,0.487141,0.322816,0.365547,1.609437e-279,1.495144e-273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38176,chr21:38345364:I_RC,chr21:38345364:I,neg,ref,901.072839,1388.354049,0.599901,12.677858,8.081503,890.896221,...,3.770434,0.000000,-0.314867,-0.166490,-0.259226,0.871231,0.584102,1.011423,8.288911e-09,1.000000e+00
38177,chr21:38345364:I_RC_alt,chr21:38345364:I,neg,alt,743.009986,1093.456565,0.496592,2.826263,0.000000,712.066055,...,10.387111,5.790755,0.063836,0.002508,0.040838,0.020828,0.017005,1.043457,1.000000e+00,1.618991e-06
38182,chr21:30327732:D_RC,chr21:30327732:D,neg,ref,370.915260,841.553676,1.114396,23.234202,18.637847,589.439600,...,40.161885,35.565530,-0.057799,0.264200,0.062951,0.506545,0.337016,0.629268,2.302252e-19,2.719383e-36
38704,chr22:50310881:D_alt,chr22:50310881:D,pos,alt,1355.235110,1261.597715,-0.101515,0.473329,0.000000,1375.781057,...,7.138922,2.542566,-0.294019,-0.157692,-0.242897,2.150959,1.447458,0.985066,1.000000e+00,2.867039e-03


In [ ]:
df_rs['snp'].value_counts()

chr17:43775212:D    4
chr17:44137009:D    4
chr17:44152736:I    3
chr7:126297864:I    2
chr5:125915685:D    2
                   ..
chr7:55544130:D     1
chr7:127142915:I    1
chr8:61397279:I     1
chr8:110533892:I    1
chr22:32793101:I    1
Name: snp, Length: 216, dtype: int64

In [ ]:
df_rs_vs_biomart = list(set(df_rs['snp'].unique()) - set(total_variant_name_biomart['variant_name'].unique())- set(total_synonym['synonym_name'].unique()))
f'{len(df_rs_vs_biomart)} rsids not found with biomart: {df_rs_vs_biomart}'

"216 rsids not found with biomart: ['chr10:102278128:I', 'chr17:44152706:D', 'chr16:3627407:I', 'chr17:44102741:D', 'chr3:195414288:D', 'chr10:45947128:D', 'chr17:44016601:I', 'chr19:36135279:I', 'chr20:25529845:D', 'chr1:25795586:D', 'chr19:50526219:I', 'chr15:39463118:D', 'chr8:110533008:I', 'chr11:4415378:I', 'chr6:32546828', 'chr9:123605711:D', 'chr1:25828322:D', 'chr17:36438743:I', 'chr7:55842921:D', 'chr10:46014867:I', 'chr11:780319:I', 'chr2:27656823:I', 'chr13:75874521:D', 'chr1:233116500:I', 'chr2:15749098:I', 'chr9:135610588:D', 'chr17:74744807:D', 'chr19:46173215:I', 'chr12:109197264:D', 'chr14:35270656:D', 'chr2:208014671:D', 'chr8:61397279:I', 'chr17:44152736:I', 'chr17:44341868:D', 'chr8:145653973:I', 'chr4:68500798:D', 'chr21:38345364:I', 'chr17:47014829:I', 'chr3:134320103:D', 'chr1:205753876:D', 'chr11:107735081:D', 'chr7:126297864:I', 'chr17:44354157:I', 'chr9:138819807:I', 'chr10:51583018:D', 'chr7:99837194:D', 'chr4:68566878:I', 'chr10:26327976:D', 'chr17:44106515:D

***
txt file maken voor query snp nexus

In [ ]:
with open ('rsids.txt', 'w') as file:
    for id in rs_ids:
        file.write(id + '\n')

In [ ]:
with open ('rsids_unique.txt', 'w') as file:
    for id in unique_rs_id:
        file.write(id + '\n')

***